This script automates inference using a fine-tuned model on the binary task of near-duplicate detection.


**Please note**:

 - the model only predicts 0 = distinct and 1 = clone/near-duplicate

 - the inputs need to be trimmed! see trim_common_html() for that

## Install necessary libraries

In [ ]:
! pip install transformers datasets evaluate
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 50.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


Mount Google Drive for data access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Model/Tokenizer and run inference

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch

current_representation = 'content'
# 'content_tags', 'tags'

tokenizer = AutoTokenizer.from_pretrained(f"lgk03/distilBERT-NDD.html.{current_representation}")
model = AutoModelForSequenceClassification.from_pretrained(f"lgk03/distilBERT-NDD.html.{current_representation}")

# load the data (just example here -> the data that the model was trained on)
current_representation_csv_file = f'modified_test_data_{current_representation}'
df = pd.read_csv(f'/content/drive/MyDrive/dataset_labeled/already_processed_modified_data/{current_representation_csv_file}.csv')
df = df[
    (df['appname'] == 'addressbook')
    & (df['state1'] == 'state141')
    & (df['state2'] == 'state239')]


def preprocess_for_inference(state1, state2):
    tokenized_inputs = tokenizer(state1, state2,
                                 padding='max_length',
                                 truncation='longest_first',
                                 max_length=512,
                                 return_tensors='pt')
    return tokenized_inputs

## List of Indices that are evaluated

In [ ]:
ix = [17417, 3253, 14797, 5461, 7546, 4006, 14109, 3277, 15545, 13385, 7219, 5158, 18649, 15362, 15219, 8296, 951, 16426, 2362, 18901, 247, 4137, 12075, 4655, 4412, 11657, 6451, 18390, 5317, 19148, 19179, 11515, 3795, 12549, 10305, 9446, 15711, 6828, 7562, 15502, 1454, 15735, 9573, 6646, 3381, 10319, 7354, 8916, 14665, 5178, 13595, 9097, 7332, 222, 3706, 4791, 5221, 9928, 10850, 10417, 7005, 12934, 5343, 7806, 18221, 17782, 4499, 9280, 1241, 3061, 18920, 17536, 13081, 4797, 17930, 3374, 13797, 11729, 4500, 14153, 4374, 13219, 19237, 17665, 18006, 7873, 5505, 9259, 436, 5107, 1693, 2050, 2812, 4807, 14254, 14913, 9823, 7551, 1846, 15103, 17776, 8710, 1851, 10268, 9960, 16919, 8665, 10641, 19195, 3195, 17203, 11986, 10959, 12601, 12022, 18358, 12867, 4529, 2485, 10191, 5130, 3439, 15853, 2251, 6640, 17380, 16677, 7123, 5777, 8872, 6834, 11161, 18473, 15017, 16695, 11425, 16859, 2697, 883, 17054, 8656, 1139, 9195, 13799, 5764, 2676, 2995, 974, 9389, 11279, 17212, 15159, 14821, 7502, 8594, 7401, 18026, 17975, 332, 9893, 18670, 6036, 18505, 18298, 6422, 4358, 5255, 11574, 19221, 17250, 15551, 1199, 14873, 14604, 53, 2171, 7900, 10175, 2353, 18371, 15727, 13204, 12244, 11787, 18836, 18570, 6763, 483, 8022, 7844, 18714, 13712, 4795, 18584, 1026, 10439, 5384, 14423, 10883, 531, 5838, 8386, 5724, 2065, 1429, 2808, 1394, 7398, 8970, 14049, 12129, 7628, 13031, 14923, 11138, 10965, 7427, 933, 5316, 9781, 6861, 7751, 7045, 11930, 8553, 2603, 18988, 10676, 15371, 11741, 11118, 8746, 6879, 16899, 2361, 13063, 15227, 4803, 9695, 17596, 15519, 18847, 1131, 10029, 4681, 18890, 1152, 14299, 13375, 14397, 6787, 17967, 4456, 7614, 6769, 7259, 7554, 13268, 3274, 17806, 5883, 9549, 15187, 67, 14047, 3471, 9292, 3705, 4465, 12521, 18675, 8729, 5934, 2580, 6997, 6961, 12579, 2383, 16805, 17759, 11076, 6368, 1156, 1285, 10997, 14765, 919, 17095, 13133, 12377, 17325, 2862, 1694, 11072, 1719, 14085, 12960, 10827, 3259, 9438, 227, 5188, 8277, 12942, 4169, 14877, 1324, 8856, 11402, 17125, 15798, 4917, 10212, 9014, 10726, 6249, 1385, 519, 13470, 16276, 18323, 17764, 15552, 15966, 8470, 10216, 19174, 2245, 9492, 14123, 18053, 12572, 4263, 5052, 1444, 17950, 1740, 7689, 6633, 9204, 16801, 9982, 1924, 17405, 4482, 18279, 291, 13060, 15112, 11175, 2823, 16371, 16044, 18113, 5049, 3245, 10363, 1527, 2395, 13157, 10302, 17579, 5383, 10749, 5310, 7488, 4057, 3872, 13987, 12538, 5980, 15728, 12077, 3501, 4901, 249, 5785, 7324, 2346, 7442, 980, 6606, 737, 7695, 12469, 3601, 5657, 12617, 6610, 1187, 1331, 10420, 16947, 8925, 6858, 11567, 4431, 18502, 12735, 18096, 3531, 8902, 10391, 1242, 5074, 12243, 13909, 17330, 17741, 19428, 4307, 11916, 14467, 4228, 10784, 9436, 10013, 15368, 6711, 6808, 7250, 7813, 1586, 10392, 11963, 10740, 13084, 7533, 18833, 6327, 17044, 9107, 11876, 6726, 8472, 11405, 13283, 17885, 1463, 3535, 4858, 14293, 5328, 4059, 11309, 17711, 4776, 12820, 14649, 1876, 13863, 10500, 9805, 14826, 8458, 9461, 6916, 2555, 6160, 12859, 11560, 5903, 12369, 7387, 19353, 4796, 5473, 2960, 5830, 7613, 14454, 19257, 12465, 15069, 9187, 9119, 3330, 3273, 15372, 5848, 10738, 12524, 7544, 340, 18759, 11919, 16238, 8988, 14120, 8843, 16981, 16576, 463, 14075, 10919, 15582, 3429, 19390, 17283, 1525, 14384, 8127, 15602, 16163, 14331, 1035, 15855, 13898, 3484, 6773, 10976, 15556, 13922, 6760, 8684, 18211, 19317, 16326, 3401, 18561, 2743, 13700, 7243, 11774, 8563, 1377, 6010, 16529, 16558, 5229, 343, 18031, 1044, 11473, 10255, 19211, 12439, 3962, 5704, 9788, 3730, 17561, 5945, 259, 12674, 2308, 15597, 6887, 19011, 341, 9543, 15952, 18621, 17477, 16383, 3071, 1439, 1017, 12643, 1196, 6949, 3372, 13675, 19124, 8465, 8039, 18061, 16311, 18289, 3199, 18549, 1584, 6015, 14678, 12541, 18771, 11260, 3311, 16631, 9415, 10072, 12771, 19198, 4062, 10843, 10519, 7435, 4176, 16004, 3435, 17301, 13557, 5723, 9604, 12291, 601, 14160, 8369, 15593, 2706, 14568, 18382, 7513, 603, 7515, 18209, 6416, 9449, 8541, 9044, 347, 13052, 3708, 13782, 13633, 1351, 7011, 19074, 15329, 11939, 9430, 8552, 10163, 5149, 159, 3094, 11697, 12297, 3996, 18718, 18287, 15668, 17093, 10099, 13786, 6749, 17445, 10032, 18717, 17022, 12457, 2315, 6278, 6431, 8739, 5709, 13393, 7664, 13959, 7826, 11082, 9785, 18973, 7220, 2528, 10654, 12871, 12563, 17114, 10494, 2558, 12831, 9010, 19106, 7499, 1757, 4754, 788, 1376, 16413, 2924, 3849, 6353, 5169, 9213, 9212, 9666, 5897, 11983, 606, 18306, 16026, 19187, 19006, 19271, 16978, 5938, 13328, 16330, 6349, 12968, 11097, 11506, 4964, 11057, 1319, 3511, 15129, 3029, 6582, 2393, 17299, 3158, 6615, 5191, 13566, 5265, 3641, 8528, 6104, 1151, 14332, 3512, 15772, 11393, 14184, 15224, 2020, 8118, 11335, 18314, 30, 219, 16942, 1857, 17988, 10640, 3475, 15484, 1969, 18952, 15789, 15436, 8749, 3449, 11135, 11077, 1080, 6563, 12182, 8591, 9657, 4198, 5793, 12254, 10752, 17637, 14926, 13608, 594, 1949, 3855, 10025, 15121, 14951, 12315, 15859, 18270, 8653, 6442, 17119, 14820, 17090, 11952, 15080, 12163, 17426, 9594, 6425, 10701, 12710, 856, 14461, 11008, 16902, 1536, 3014, 5101, 10339, 5593, 4010, 17927, 725, 4053, 2115, 19051, 15286, 859, 15449, 13372, 16768, 12168, 8214, 6271, 19230, 5408, 17110, 8498, 3909, 6309, 18147, 15086, 11201, 18716, 18789, 285, 12392, 16391, 1383, 4445, 10227, 18454, 3340, 17179, 17951, 4343, 2945, 14040, 15707, 15472, 10675, 3054, 5502, 2980, 18572, 19098, 8832, 8897, 18464, 3128, 5706, 15831, 17012, 9569, 16518, 15259, 4707, 15790, 15890, 2123, 10157, 13285, 6240, 336, 15358, 10842, 13046, 3526, 9983, 3775, 1478, 2754, 4124, 11674, 1830, 13783, 17257, 8278, 2386, 19294, 5824, 10650, 4595, 9424, 17353, 9358, 5155, 17580, 17751, 13074, 8233, 8445, 10468, 9089, 2771, 13966, 3105, 3350, 8474, 6395, 2504, 5290, 15750, 6758, 12386, 17924, 15721, 18378, 3064, 5056, 9626, 9251, 1356, 3917, 14833, 2726, 11016, 14839, 12159, 14288, 2946, 11191, 120, 9965, 4815, 6987, 7763, 17460, 4448, 4110, 10282, 17400, 17267, 12482, 18775, 3292, 4695, 5271, 3620, 11326, 10063, 2048, 276, 4792, 19280, 12512, 7308, 3934, 16543, 10739, 9231, 17864, 15235, 11378, 14347, 6071, 7691, 4245, 8289, 18624, 10719, 3427, 14004, 18286, 6475, 9849, 10186, 8732, 7924, 7715, 14151, 12654, 19440, 16927, 19010, 1651, 15483, 17649, 9969, 9819, 5148, 6313, 11805, 4211, 19196, 13862, 11199, 11106, 2632, 19326, 13804, 7116, 3877, 12975, 5517, 19330, 7919, 12613, 11244, 739, 16918, 11421, 17563, 11818, 11273, 7359, 10938, 13195, 12854, 13817, 11548, 11286, 18607, 9895, 16434, 17291, 8751, 4192, 2039, 14044, 10504, 9381, 10782, 14069, 17949, 11837, 8516, 8411, 1078, 1003, 12044, 3685, 6813, 4686, 9217, 18442, 2427, 7575, 13550, 12232, 851, 4317, 16656, 8187, 14186, 11391, 6546, 10541, 11641, 14479, 3239, 9881, 2280, 13222, 15361, 119, 1014, 2044, 9255, 10341, 17269, 10379, 15381, 11882, 13647, 2947, 2418, 13015, 12887, 1859, 1461, 18932, 12503, 3559, 7464, 14818, 15704, 1934, 18928, 1464, 14477, 6277, 344, 103, 10516, 13464, 9767, 6500, 3231, 10555, 3826, 5329, 8756, 17734, 17542, 6334, 7991, 2354, 1730, 16923, 18090, 717, 7814, 6504, 12491, 6143, 1214, 10266, 14472, 17215, 13129, 5421, 12792, 8290, 7759, 4615, 8681, 17374, 2902, 1111, 12427, 6757, 6325, 15980, 10931, 11268, 10639, 6455, 14244, 16511, 13099, 15635, 3667, 2097, 14072, 7945, 12103, 13932, 19159, 5417, 4820, 652, 8964, 7496, 11595, 16353, 4903, 874, 6158, 13079, 3457, 14430, 19339, 14621, 18401, 17646, 11049, 5054, 5983, 19161, 15324, 18822, 10229, 8865, 2907, 4067, 14055, 19228, 1425, 3972, 3001, 4650, 8502, 11267, 13092, 4856, 4548, 18470, 4074, 9533, 9783, 10487, 13739, 5599, 8249, 9048, 18439, 7637, 10865, 4109, 4600, 1249, 19434, 9738, 4393, 360, 15173, 15674, 12558, 3295, 17667, 17774, 16260, 8227, 6112, 411, 12997, 16908, 9540, 7959, 7030, 7041, 5976, 6284, 14704, 8475, 4721, 10570, 5212, 14439, 8305, 12628, 6976, 4732, 17248, 13960, 7535, 877, 10946, 11328, 9713, 17300, 12547, 4078, 12930, 17588, 15375, 5965, 17161, 298, 454, 15089, 4985, 18252, 5836, 14988, 19197, 6698, 16684, 16643, 2934, 6136, 1244, 862, 18169, 8269, 12238, 5091, 15531, 5120, 12978, 9257, 12571, 5186, 14449, 1166, 19079, 8364, 14289, 11493, 6407, 18577, 8918, 10539, 8712, 12036, 11733, 11063, 13781, 4905, 8837, 18966, 3432, 12609, 12323, 13108, 1058, 11375, 8955, 15745, 14329, 3772, 16405, 6579, 1745, 5604, 18803, 870, 1344, 3420, 6119, 378, 13122, 17681, 15216, 9162, 15912, 3466, 15770, 12749, 11166, 17983, 4624, 17670, 3657, 18160, 6609, 14654, 17430, 8199, 3265, 14367, 684, 18082, 4991, 8659, 14366, 10571, 6576, 13109, 3207, 9507, 17337, 13860, 18941, 2838, 7828, 15598, 18712, 13716, 12286, 650, 14740, 8530, 4148, 19229, 2668, 1985, 1892, 17220, 13259, 16265, 1759, 5210, 15935, 13083, 12399, 10089, 19421, 10474, 6244, 19026, 17688, 10496, 11822, 16028, 10368, 6957, 10669, 11445, 6100, 13291, 15913, 16815, 9242, 15055, 15053, 13003, 12714, 5170, 11274, 18272, 5215, 4583, 5782, 12523, 7768, 11180, 10125, 4082, 4572, 11272, 6864, 17620, 14480, 146, 3109, 4167, 2103, 5813, 10289, 962, 13890, 13818, 16758, 5826, 5570, 9062, 14286, 3306, 13135, 908, 15305, 11528, 13202, 3889, 2721, 15342, 875, 11101, 15643, 19287, 5870, 9834, 19302, 18387, 16691, 11394, 3807, 19035, 11810, 2291, 8167, 6565, 4309, 10695, 3789, 4378, 10122, 13401, 7934, 7634, 8230, 16711, 4633, 4854, 5208, 16170, 16069, 2964, 7343, 13678, 17235, 5936, 15152, 18508, 9645, 18738, 3462, 6467, 194, 350, 6478, 5528, 4612, 2206, 17804, 3973, 2997, 12213, 11648, 1982, 17636, 8327, 18188, 115, 16574, 2621, 2005, 1206, 1494, 14548, 13130, 13038, 514, 6547, 7133, 6157, 18692, 17792, 1979, 15332, 2196, 10715, 13391, 4716, 17448, 18583, 5567, 5562, 503, 7633, 10575, 11795, 17354, 8323, 14443, 15926, 2515, 8695, 10950, 15374, 639, 17540, 2990, 17632, 2146, 1019, 1123, 18982, 18736, 14799, 16600, 13774, 10729, 10055, 12488, 17138, 4433, 5117, 15825, 17323, 17762, 6675, 7897, 5792, 7412, 7825, 12829, 1986, 18297, 2435, 5944, 16177, 4944, 14102, 2188, 10443, 12595, 13701, 923, 10263, 10431, 18326, 14698, 3585, 14361, 13016, 13050, 6499, 3545, 13437, 13152, 17492, 16843, 10754, 138, 8705, 11200, 8354, 15732, 14751, 2404, 13021, 9892, 16777, 11970, 3097, 1619, 587, 11649, 1105, 18869, 7514, 8284, 4577, 4981, 14201, 1662, 7911, 9625, 6252, 5873, 6215, 4293, 1945, 13431, 6022, 1718, 481, 373, 12476, 12477, 8148, 7787, 8347, 6142, 8403, 532, 11007, 13638, 11280, 10230, 11420, 16409, 4145, 3644, 3015, 18645, 10128, 13719, 10733, 10741, 5007, 16630, 10014, 15064, 6453, 1130, 14154, 13008, 14743, 7983, 14641, 5401, 16748, 5683, 293, 274, 1398, 1141, 10643, 5322, 12108, 12802, 7336, 161, 12236, 4438, 11171, 1481, 16683, 1914, 4781, 5899, 10712, 6510, 17629, 17995, 16045, 10708, 17546, 17926, 4372, 13293, 7330, 2013, 11060, 15003, 19299, 17413, 9003, 3976, 11333, 4012, 14341, 9757, 10148, 14165, 13045, 7853, 158, 11022, 18872, 7481, 3508, 15911, 12842, 19366, 1016, 12436, 14488, 17066, 19203, 18685, 1786, 9479, 14970, 4657, 12143, 13976, 4626, 2952, 1049, 15720, 11924, 2416, 10446, 11786, 312, 10700, 8795, 6365, 15454, 15799, 2805, 18367, 7969, 1007, 14243, 14703, 1169, 5082, 12343, 6634, 13436, 13, 10020, 10653, 74, 6464, 14522, 11703, 10093, 18210, 9443, 9565, 18248, 6539, 5687, 805, 4200, 9192, 15349, 4001, 10008, 13602, 7944, 13495, 4275, 19101, 10081, 18376, 2839, 16750, 9911, 6974, 17455, 7261, 2321, 16123, 16009, 381, 8757, 19264, 17329, 1013, 490, 15413, 6262, 17355, 5018, 890, 16920, 15495, 15554, 10304, 7084, 14623, 9509, 13292, 15886, 10554, 13570, 14575, 11298, 12897, 16811, 1401, 4667, 5788, 7229, 5432, 7300, 11339, 17993, 16272, 7476, 3383, 3999, 9460, 11760, 5551, 8913, 3248, 16928, 13096, 17032, 14456, 14073, 17025, 7370, 13769, 7734, 15687, 14111, 1262, 1022, 5632, 7287, 5459, 9609, 19144, 3150, 15511, 12813, 9177, 13916, 1628, 10608, 15517, 16128, 10390, 9294, 9735, 7237, 945, 11337, 3444, 11488, 4539, 2930, 324, 17818, 7526, 3571, 15455, 15467, 3042, 17590, 19368, 3945, 9861, 1122, 8852, 4391, 3716, 2218, 18465, 3635, 3270, 12953, 5019, 4244, 2958, 4737, 17602, 7693, 17060, 16428, 3017, 766, 2279, 8714, 13546, 18154, 18628, 2494, 7234, 5023, 17940, 11153, 12551, 7196, 16473, 723, 17034, 4760, 6955, 7954, 1235, 11111, 14957, 13669, 8625, 6305, 9293, 414, 9595, 17272, 13673, 13351, 6894, 18861, 11713, 2781, 9256, 13995, 16080, 11671, 17641, 16858, 18540, 17475, 4140, 5602, 9922, 14024, 11776, 5094, 2467, 7566, 12818, 1218, 16696, 7268, 1966, 4118, 16467, 5044, 18312, 9718, 17083, 12068, 3188, 8698, 3941, 7015, 1796, 7475, 6926, 5182, 14503, 4601, 19394, 9080, 17366, 4653, 12493, 10647, 13801, 9538, 872, 1962, 878, 6723, 12082, 15762, 18910, 14016, 15659, 5183, 3524, 11087, 7876, 994, 5543, 16674, 4384, 16609, 18309, 12796, 1033, 15077, 18379, 12371, 12924, 8841, 7767, 7445, 10460, 14494, 16038, 6835, 8342, 11083, 9051, 6921, 10009, 5787, 9932, 384, 8038, 16938, 10167, 7063, 12495, 16553, 1286, 7746, 17100, 9641, 8624, 9410, 17479, 4194, 12606, 11993, 3884, 284, 8602, 7463, 8250, 15386, 2283, 6140, 2651, 2655, 4380, 8514, 279, 10277, 6333, 3448, 12545, 9202, 10374, 4351, 10034, 13056, 8408, 1802, 6958, 18616, 6791, 10286, 4880, 10187, 11117, 11559, 4108, 15758, 11442, 744, 9173, 16, 1535, 14063, 18294, 8192, 5929, 14136, 14387, 5655, 7936, 13651, 16317, 16008, 8241, 19081, 2369, 15404, 8217, 18933, 167, 9026, 11241, 7988, 6497, 8409, 9018, 10219, 8272, 8321, 543, 8983, 5073, 17347, 10905, 16586, 2975, 754, 11046, 9956, 15314, 13380, 84, 14876, 16896, 16638, 2101, 12330, 15947, 8658, 8285, 13288, 5488, 64, 16288, 9246, 11923, 4602, 13216, 1661, 12825, 16454, 3010, 19113, 14906, 9296, 724, 4280, 3072, 16088, 19184, 12263, 1984, 14801, 15444, 15024, 6190, 297, 16667, 7274, 8982, 2850, 18713, 13480, 13066, 13463, 2665, 18245, 713, 5180, 13737, 13432, 433, 10799, 12591, 15988, 12239, 15507, 9931, 16139, 7291, 1647, 15876, 13983, 1446, 10035, 2368, 14815, 8951, 8839, 17251, 1946, 467, 12619, 6888, 18600, 7840, 2240, 8644, 9719, 4183, 173, 14034, 10718, 11894, 12246, 4428, 790, 6548, 4928, 8056, 3082, 10137, 13041, 11456, 18413, 689, 1588, 18155, 7342, 4731, 18363, 10022, 14412, 7329, 1316, 2883, 8674, 13952, 3358, 2929, 11250, 415, 7357, 16345, 16145, 2923, 8965, 882, 18489, 18626, 5917, 14458, 3504, 8360, 18141, 16503, 8744, 3049, 2857, 18827, 9222, 10458, 2561, 5053, 15979, 1258, 17394, 8995, 15130, 15424, 3887, 3187, 2096, 3155, 5679, 9000, 1994, 10086, 13787, 16022, 14949, 1433, 12120, 10559, 6899, 3725, 6319, 5738, 16985, 11910, 14856, 17921, 8669, 7981, 10303, 8232, 15466, 366, 14708, 16953, 6849, 4015, 18874, 2214, 13963, 6079, 1770, 6391, 3437, 17344, 8945, 8395, 7335, 3185, 16181, 19252, 6233, 15719, 17775, 12602, 6301, 46, 12575, 3954, 5176, 15922, 5283, 12789, 11152, 12965, 3704, 4105, 2881, 5124, 3712, 12847, 5748, 16648, 14510, 8483, 6545, 4639, 16856, 18253, 8549, 10557, 7832, 12780, 1848, 9095, 16722, 11415, 12099, 17201, 7071, 1738, 17139, 2492, 2129, 6928, 18580, 1416, 213, 3832, 18028, 14410, 14100, 17726, 4621, 12447, 18703, 6210, 3527, 7736, 1702, 3610, 6521, 14272, 8547, 10631, 14018, 3133, 10012, 7185, 17336, 16029, 11403, 7645, 10402, 16736, 13594, 4801, 18897, 9677, 2893, 4859, 15521, 16151, 2557, 18254, 911, 12943, 4090, 5190, 2855, 15557, 18375, 3777, 11734, 5705, 18448, 10893, 11156, 16477, 13512, 18266, 18527, 15636, 18552, 16944, 9825, 11643, 13364, 11302, 12433, 14999, 13610, 368, 2900, 9166, 14628, 6414, 1626, 12660, 17018, 16781, 14889, 15804, 13450, 7248, 8141, 394, 4371, 19316, 7157, 818, 795, 5727, 14313, 18707, 10544, 18195, 17900, 13311, 16169, 10124, 15836, 681, 264, 1266, 6685, 2412, 6923, 5113, 15974, 5832, 15572, 11881, 1877, 3469, 10929, 7211, 8298, 1472, 12146, 3115, 6315, 2852, 18922, 470, 13785, 2815, 3281, 13876, 16110, 16186, 11998, 8313, 1038, 3212, 5400, 6553, 9232, 1452, 10469, 12492, 16620, 18299, 6643, 2735, 5833, 1448, 4255, 11395, 7593, 9945, 4527, 17675, 9369, 16873, 12245, 18551, 5217, 2519, 10346, 12067, 17074, 12689, 17488, 18016, 9032, 10681, 9113, 2656, 5270, 1569, 14702, 14800, 435, 18384, 369, 3215, 11155, 1571, 18709, 11618, 14969, 14750, 14939, 5930, 3145, 8696, 17071, 5725, 14530, 486, 7643, 16418, 12528, 2939, 14964, 10225, 7659, 6844, 5364, 15787, 16173, 5964, 8579, 4704, 13493, 15073, 5584, 9782, 4673, 15583, 11736, 6897, 12827, 8816, 3228, 16528, 7880, 10611, 6306, 5617, 573, 13153, 9485, 16730, 12158, 1191, 18894, 14515, 4493, 15360, 9556, 11576, 7036, 879, 3613, 14504, 17752, 8271, 14831, 4442, 11792, 4121, 13298, 19405, 16549, 6330, 9133, 8190, 5238, 7487, 3693, 15252, 17863, 18637, 12442, 15021, 18665, 17868, 11762, 12741, 10356, 9755, 3776, 747, 2436, 6373, 6144, 7591, 15347, 12356, 4181, 13087, 13396, 10033, 2128, 15, 9412, 16888, 12223, 10656, 17031, 16675, 12178, 1256, 16997, 19131, 10509, 17917, 11221, 12363, 10909, 1009, 7127, 246, 16408, 2461, 3553, 2891, 15138, 5747, 8608, 9656, 10003, 15071, 2669, 2356, 11497, 11843, 18938, 10837, 2917, 8850, 14841, 3970, 12430, 9169, 2554, 4786, 482, 771, 2396, 5430, 4710, 5817, 17920, 894, 19282, 9134, 10599, 18937, 9227, 15109, 19456, 13839, 12199, 18421, 5230, 4187, 8672, 12957, 3317, 5479, 576, 9851, 12335, 17042, 9876, 14103, 7446, 16860, 17184, 362, 16184, 9270, 14874, 9873, 10169, 10674, 11821, 6793, 11974, 195, 15569, 6360, 1983, 661, 14225, 4077, 2539, 14296, 8650, 4337, 13013, 8572, 12289, 1881, 14182, 12145, 18260, 15628, 2425, 11773, 10218, 17716, 2015, 16909, 5515, 6869, 5851, 6393, 10523, 385, 9054, 17349, 6786, 10220, 7172, 2636, 12759, 1644, 14711, 16680, 14133, 13559, 12357, 5522, 7818, 7482, 636, 4, 18293, 5588, 9216, 16575, 3446, 8935, 13684, 8525, 13516, 14849, 1981, 14357, 14345, 12557, 1120, 9547, 18606, 4241, 6114, 1882, 11068, 8435, 4477, 2601, 7091, 4373, 11246, 9929, 3810, 280, 9238, 9104, 9288, 6718, 1150, 11802, 17943, 19300, 13028, 17469, 9263, 2289, 942, 8904, 17948, 8494, 15864, 14021, 2778, 6674, 4327, 2987, 2303, 16094, 6109, 9997, 10958, 6796, 14358, 12982, 9064, 4217, 5585, 8352, 7265, 3481, 16343, 5291, 9269, 17395, 15271, 6397, 8817, 6823, 1491, 4000, 9512, 2976, 1069, 18987, 105, 5233, 17158, 11400, 6007, 16841, 15814, 18046, 11453, 19147, 9348, 17356, 17501, 627, 12190, 9108, 9121, 814, 16093, 13355, 10422, 12237, 12056, 10372, 15872, 19034, 16590, 631, 18187, 2254, 12186, 18431, 10476, 12440, 16836, 10354, 5134, 1369, 16864, 6470, 9717, 11516, 6274, 2573, 17812, 2688, 9521, 13683, 12855, 17285, 11002, 10101, 6049, 57, 2035, 10166, 10867, 391, 8237, 18802, 15504, 9721, 16653, 8550, 15262, 10127, 9065, 6083, 1298, 750, 6744, 19135, 3858, 18410, 4353, 13469, 6063, 16459, 16803, 2477, 17939, 3006, 11928, 18582, 5744, 14178, 16362, 14374, 10403, 15181, 14152, 4491, 12511, 18403, 17897, 5281, 18895, 13806, 16904, 4645, 1539, 18823, 11169, 19381, 14008, 18785, 7884, 2984, 10428, 6795, 14892, 113, 15827, 487, 7111, 1698, 5690, 14349, 3722, 11897, 8771, 18840, 803, 17260, 5403, 12662, 10092, 11462, 18970, 4993, 4076, 12312, 1445, 8673, 676, 12234, 17345, 3361, 15051, 8206, 18337, 13525, 15970, 5639, 8422, 6561, 3086, 13156, 15807, 100, 2301, 13209, 9262, 5433, 3434, 8338, 6295, 11723, 3860, 6850, 861, 7692, 12346, 16514, 17968, 9244, 15914, 10195, 12060, 1381, 2899, 3380, 9330, 11205, 13878, 18194, 9741, 19388, 6042, 11584, 893, 1839, 1541, 16199, 6246, 16641, 15290, 9890, 19411, 14694, 10262, 8614, 18904, 10899, 10336, 18660, 8390, 18503, 19340, 5031, 9297, 4540, 12042, 16510, 5156, 10535, 4868, 479, 7640, 4489, 4980, 8372, 8978, 8388, 11720, 18207, 10190, 17185, 10206, 18663, 2948, 830, 13116, 12362, 6922, 4350, 7586, 3995, 14705, 16464, 2110, 1186, 4037, 7217, 3558, 4035, 16494, 2032, 11031, 944, 8654, 11504, 8010, 7014, 14764, 7143, 5336, 14048, 1550, 11247, 6361, 1109, 2998, 2453, 16835, 3836, 6605, 17838, 6087, 15460, 8457, 7082, 4772, 7898, 12059, 12755, 1578, 5825, 15672, 11901, 3625, 12964, 8524, 13545, 17016, 16314, 9821, 16333, 11558, 5070, 17453, 11549, 5843, 7192, 15778, 14666, 5160, 6911, 12678, 19004, 5246, 522, 5662, 9787, 4058, 1144, 17813, 16657, 17677, 3479, 9685, 8642, 1712, 13734, 7677, 14646, 4088, 15630, 16210, 4473, 15079, 5205, 8600, 12814, 12104, 14898, 15991, 8998, 18560, 4455, 18576, 9517, 10084, 13422, 570, 9514, 18999, 13674, 9714, 18004, 9578, 13179, 16324, 8846, 2036, 14747, 9870, 701, 4155, 17925, 2643, 11066, 7609, 1744, 14565, 12038, 11909, 10866, 2352, 15061, 7995, 16983, 283, 1037, 12472, 3403, 18597, 11777, 13895, 11300, 14495, 11842, 1680, 2641, 15433, 10984, 12337, 17733, 5615, 16971, 10900, 8011, 8890, 3114, 14196, 14701, 9159, 1342, 4812, 6379, 7450, 8512, 8264, 3742, 8365, 10467, 7159, 1188, 4475, 11354, 16567, 11044, 4659, 16356, 11716, 12920, 14866, 4017, 7630, 17369, 16252, 12125, 14114, 9619, 8845, 8072, 15897, 5359, 97, 2052, 474, 8569, 9466, 9071, 6734, 1008, 18324, 8652, 10475, 19002, 19389, 17137, 1727, 11879, 18949, 2698, 7417, 15981, 4168, 19020, 2403, 15885, 18728, 11331, 2516, 10423, 12999, 17281, 9114, 9818, 11854, 16991, 13349, 15919, 14497, 18060, 3523, 8144, 1183, 8120, 10132, 6011, 7730, 7781, 2784, 17233, 9799, 3580, 17151, 11177, 238, 622, 15343, 11827, 13777, 12449, 7942, 4929, 6186, 15335, 16052, 5179, 6058, 6862, 16137, 11433, 1493, 12772, 14468, 16868, 10083, 17271, 3684, 5600, 18216, 8204, 900, 5649, 11042, 7660, 8975, 3980, 9405, 6874, 15122, 7619, 582, 17268, 2541, 8493, 6662, 4698, 11399, 979, 10603, 157, 3910, 7644, 40, 6788, 709, 8987, 552, 13535, 12422, 18329, 4139, 17263, 7088, 8055, 3332, 15192, 12201, 2979, 15587, 1265, 13000, 10506, 961, 3988, 14511, 13398, 14157, 2821, 8171, 14030, 6211, 12184, 5137, 19049, 16789, 16886, 17699, 11146, 16735, 18858, 6328, 8565, 12358, 15196, 7426, 10362, 6599, 3820, 91, 14444, 10456, 9444, 2378, 10710, 12127, 2572, 15041, 9642, 4592, 7952, 12402, 3375, 9283, 310, 3616, 1592, 11862, 16623, 15620, 6488, 8317, 12332, 18462, 15170, 16098, 315, 19447, 18995, 550, 17595, 8646, 7550, 16685, 5279, 15143, 7673, 1996, 17558, 907, 11271, 3519, 4660, 12463, 5889, 10884, 10160, 17516, 18325, 6522, 4614, 8570, 6735, 6948, 17785, 5092, 3335, 2463, 6866, 14339, 8946, 18374, 5736, 13776, 14303, 4106, 15995, 772, 9024, 3743, 13118, 13715, 17169, 6392, 7345, 15318, 7421, 3142, 3557, 13612, 15463, 15849, 3931, 18994, 14197, 12894, 1602, 10657, 10934, 16189, 1178, 8860, 15447, 16591, 16141, 16485, 17586, 13864, 2702, 13968, 11449, 4995, 868, 12583, 2611, 17672, 3002, 1620, 10134, 17577, 12922, 5293, 15740, 9131, 5819, 16491, 15652, 9727, 5933, 986, 15385, 18269, 12293, 6394, 17889, 16857, 15708, 4890, 4671, 5920, 2413, 5996, 18441, 936, 17802, 4571, 17687, 2490, 1818, 12261, 10627, 16281, 17117, 14729, 9974, 7950, 16874, 10358, 15941, 56, 12327, 14205, 10831, 10261, 19115, 13569, 11785, 12616, 13513, 17073, 16223, 16095, 10713, 8427, 18747, 19117, 2563, 9962, 11701, 8418, 18398, 2986, 10580, 14907, 3157, 3260, 7908, 8453, 15688, 7117, 1060, 4662, 16766, 8489, 16316, 1281, 11873, 4392, 16625, 988, 9028, 18678, 11846, 15908, 15647, 828, 4196, 11914, 992, 9084, 14938, 18352, 15976, 9495, 13565, 6688, 13680, 244, 267, 768, 11718, 1526, 19243, 17652, 18201, 2904, 1264, 2160, 19141, 12283, 9471, 2615, 4541, 9135, 5287, 6960, 7223, 5245, 19318, 12589, 2819, 4603, 4055, 12723, 2677, 356, 18695, 1568, 110, 14393, 6469, 817, 14301, 16524, 13192, 6964, 19119, 13707, 17431, 15964, 9400, 10050, 16827, 12712, 18330, 15631, 19386, 14079, 11985, 6684, 643, 8040, 10904, 17865, 18767, 9990, 6053, 7003, 423, 14963, 132, 271, 7180, 1643, 19418, 7376, 6781, 5984, 16006, 2211, 15850, 16892, 12029, 930, 15553, 9253, 13201, 4838, 12865, 1100, 18437, 12612, 7047, 7608, 7144, 6205, 8636, 11989, 11038, 1451, 4071, 424, 4947, 10056, 1400, 4869, 13187, 15056, 11163, 6016, 15312, 15317, 9070, 9791, 15198, 18770, 16668, 11115, 6004, 1641, 3575, 10385, 15238, 12117, 9210, 5784, 16057, 12260, 5860, 15140, 9963, 6524, 5150, 16678, 7051, 2956, 3461, 7215, 18986, 5541, 1513, 3441, 24, 1185, 16097, 8085, 2102, 8901, 700, 9365, 10326, 13376, 9350, 8455, 9404, 12766, 12270, 9942, 12423, 6099, 1476, 3302, 3357, 9336, 16509, 959, 10438, 15803, 8367, 19075, 6450, 10748, 10376, 11295, 10825, 6220, 16755, 8914, 18436, 19052, 18027, 2760, 6098, 15213, 18879, 4201, 8593, 17056, 6672, 17600, 13280, 12428, 625, 4287, 498, 15036, 7176, 19469, 9267, 823, 19097, 15917, 10797, 15928, 13460, 16987, 4487, 16232, 16797, 9815, 13618, 7224, 866, 14947, 14696, 15293, 16762, 12977, 4332, 12757, 12322, 1538, 8818, 12089, 5534, 5979, 8339, 579, 6192, 5123, 8618, 9261, 9545, 13018, 10836, 17360, 14207, 11754, 10542, 4466, 12931, 14268, 12707, 4304, 7131, 4811, 5922, 17650, 16290, 1834, 19324, 17651, 11812, 13583, 12973, 9753, 17368, 14864, 18419, 5538, 1453, 16960, 6856, 9090, 13297, 10484, 16066, 9016, 3676, 11412, 2513, 18153, 1944, 11183, 14902, 16358, 16954, 15098, 3141, 16793, 4243, 8201, 6889, 12562, 5601, 10757, 14922, 2957, 7774, 541, 17576, 12224, 9610, 7109, 16376, 4219, 4844, 12387, 5226, 301, 6836, 14027, 16570, 5106, 16070, 18797, 10002, 6490, 17884, 102, 17094, 1197, 7548, 6512, 13524, 740, 11410, 4400, 13333, 3068, 18212, 11976, 18166, 8599, 12212, 2773, 12703, 11814, 9503, 16087, 1410, 7972, 7282, 13997, 1297, 13412, 12041, 15100, 18883, 11204, 704, 15422, 15147, 7432, 19327, 18146, 14045, 5714, 2525, 18036, 9792, 14804, 10761, 5299, 898, 16688, 7086, 8692, 10256, 17404, 4357, 7162, 11327, 5065, 6619, 15284, 17630, 10335, 12566, 14601, 843, 11585, 17262, 9252, 13605, 9455, 4161, 18152, 1386, 19149, 4472, 10193, 12729, 19080, 3079, 10387, 2290, 18500, 407, 17962, 4973, 163, 7538, 2476, 8742, 205, 8745, 13802, 18925, 12024, 720, 2462, 15440, 17202, 7798, 15050, 17543, 4369, 956, 7080, 11483, 5038, 2478, 11819, 15028, 17721, 5197, 6517, 3669, 3012, 152, 8096, 13604, 12600, 17616, 9750, 10233, 2537, 2113, 4875, 18535, 13992, 11409, 12451, 17592, 18725, 14673, 891, 10452, 15592, 6263, 17825, 8143, 12401, 13982, 13345, 11979, 3497, 18545, 8820, 3422, 8963, 9076, 17552, 707, 18630, 4580, 8506, 10829, 15364, 3562, 10248, 9276, 2648, 10491, 70, 14046, 12018, 14137, 9005, 7707, 6742, 17124, 13203, 4345, 11964, 4699, 1776, 17898, 4079, 15453, 7467, 12140, 17979, 4872, 9996, 12815, 6066, 15207, 10408, 18143, 727, 3164, 952, 7221, 14523, 10605, 5818, 4460, 16429, 619, 3291, 11602, 6783, 18014, 160, 9957, 17427, 1741, 20, 16875, 10998, 6298, 2885, 7403, 9347, 18038, 10371, 14185, 2387, 9206, 3689, 12913, 1215, 1624, 5080, 9337, 8509, 17437, 3882, 3920, 6446, 9464, 18406, 8921, 7946, 2264, 15114, 16334, 3116, 7195, 16917, 339, 16182, 17247, 9368, 10566, 10597, 14932, 10600, 824, 19134, 13217, 15295, 10473, 1856, 2391, 1517, 8641, 15471, 3414, 17160, 13933, 14671, 3852, 1254, 7761, 1638, 621, 15118, 295, 12414, 14235, 14642, 10450, 559, 17348, 18040, 4424, 10316, 16746, 6155, 15932, 3363, 14945, 7647, 14473, 7416, 7254, 19015, 10298, 14589, 7687, 11207, 4269, 15157, 10688, 4510, 2744, 17817, 10816, 2312, 14326, 18469, 9366, 5013, 12128, 535, 4414, 18322, 1360, 16692, 17130, 19412, 5731, 16136, 9313, 11784, 15241, 2667, 2716, 2167, 2966, 15043, 13803, 6794, 3390, 1780, 2385, 6260, 10858, 5139, 18222, 1503, 5949, 16838, 14041, 7245, 7694, 10637, 16159, 13552, 15419, 13088, 15430, 10171, 10287, 9098, 17942, 10802, 7675, 8575, 13313, 14127, 193, 15892, 5554, 2564, 12661, 10507, 10270, 14208, 17750, 9120, 3276, 13366, 4289, 11604, 2445, 8419, 15733, 5415, 13049, 1669, 15710, 9943, 7246, 11878, 4703, 2256, 16569, 3299, 13924, 973, 7266, 6459, 18271, 8788, 14514, 11174, 66, 15858, 1276, 17343, 6110, 15925, 9247, 4395, 1418, 1028, 1868, 10142, 9023, 7769, 15877, 6677, 12592, 3337, 16689, 5733, 12406, 17608, 18839, 4016, 656, 3394, 6194, 11808, 17656, 5516, 9172, 16245, 8484, 1075, 2076, 2710, 15938, 3412, 8398, 4519, 8366, 888, 14146, 8919, 15923, 14541, 8320, 7648, 6165, 16460, 6251, 3897, 5378, 8924, 14217, 8571, 7968, 13017, 1943, 13564, 2720, 5823, 12282, 14442, 5750, 9491, 11317, 4734, 5656, 14370, 3786, 15883, 6051, 17393, 2582, 4654, 3307, 1761, 14749, 15679, 9487, 3102, 13741, 6872, 3179, 8107, 8373, 10897, 18617, 18886, 17680, 308, 11689, 7603, 9328, 19276, 6092, 8649, 2074, 17198, 1259, 12241, 5729, 4505, 12730, 15257, 17545, 8480, 5726, 9914, 1318, 17, 15760, 14630, 8476, 16490, 405, 17559, 10543, 17737, 7473, 4881, 14484, 11380, 17757, 1968, 12559, 15845, 7333, 13529, 4170, 7364, 3359, 15618, 658, 13855, 11782, 19140, 3589, 2624, 19171, 11783, 5650, 9039, 4157, 6466, 14537, 695, 2846, 2571, 19063, 18215, 7286, 19070, 14755, 6055, 4397, 13180, 15646, 12546, 15432, 5243, 15595, 16058, 3792, 4609, 4774, 16974, 17167, 8005, 6775, 8822, 1529, 19319, 7859, 6598, 12754, 2579, 3346, 1101, 2751, 6687, 175, 5699, 19297, 15936, 7136, 1018, 17296, 1428, 3048, 12702, 15179, 17259, 16500, 12935, 17036, 12790, 4782, 8334, 17456, 5646, 14135, 19087, 18841, 901, 2230, 7560, 778, 9218, 2649, 10981, 16243, 16014, 5821, 6905, 14713, 13241, 1200, 1960, 16318, 2344, 8335, 17011, 11503, 19431, 11888, 7020, 4464, 1153, 9029, 15673, 19413, 18867, 7079, 4216, 18590, 6562, 16400, 16901, 14584, 11149, 12672, 12803, 11874, 554, 8655, 18625, 5257, 7823, 5110, 6939, 15040, 13540, 8392, 11108, 5868, 7155, 3622, 14901, 11173, 14124, 14946, 15044, 14631, 8968, 2326, 3662, 18118, 18581, 19076, 2761, 18275, 8492, 10838, 4102, 12220, 16651, 8410, 8887, 7326, 8755, 910, 5859, 2449, 16693, 3459, 9789, 7547, 16621, 15451, 7026, 5219, 17834, 3650, 17333, 6287, 12636, 19061, 10924, 6591, 16442, 121, 17436, 7474, 11019, 18954, 2003, 13481, 13001, 1824, 16779, 8758, 11397, 1676, 15175, 15852, 11975, 8842, 17664, 10245, 11262, 12336, 9839, 8582, 14202, 2818, 14193, 5768, 9237, 13689, 13911, 7975, 5225, 1498, 8737, 15953, 19404, 55, 18251, 19292, 18008, 3757, 10005, 5520, 4174, 8385, 16433, 6638, 4300, 2954, 13821, 2142, 5611, 13369, 13346, 14627, 17780, 4314, 18578, 17292, 19289, 6346, 5525, 16970, 13183, 809, 13441, 5998, 3364, 3222, 831, 6876, 14697, 1234, 17724, 19164, 11297, 11349, 7349, 10121, 14083, 6567, 14380, 17388, 7980, 1980, 17039, 9399, 10520, 12632, 2759, 4744, 18853, 6812, 6741, 19116, 14118, 10317, 16469, 3336, 179, 12994, 1919, 7745, 18307, 2695, 11283, 16229, 11346, 10756, 730, 5985, 252, 12561, 10236, 1409, 14287, 9240, 2681, 1334, 9346, 3148, 1282, 5477, 13190, 8740, 8784, 8299, 5105, 18795, 5341, 7295, 2647, 1828, 9033, 5504, 4830, 18765, 17197, 12786, 87, 14080, 11383, 6347, 15486, 6381, 8032, 1480, 7718, 14934, 19417, 16011, 199, 16431, 17035, 9241, 6169, 15457, 17717, 13635, 6405, 8836, 1606, 18386, 5041, 8444, 10006, 15172, 6631, 3851, 15405, 17101, 8331, 11126, 13303, 16706, 18815, 4598, 485, 15773, 17242, 17106, 15819, 8750, 11608, 4970, 4588, 11265, 13768, 3991, 14780, 9052, 16530, 13745, 11374, 7916, 13137, 15724, 14773, 3192, 5743, 18972, 18062, 16604, 6329, 4877, 5919, 1590, 10061, 19279, 14683, 14147, 8915, 10774, 1126, 7915, 15683, 11517, 2186, 15020, 18120, 9930, 19356, 13025, 1925, 4026, 272, 13147, 13840, 5220, 3975, 12257, 9160, 16716, 2307, 12565, 14602, 1598, 1835, 18244, 12692, 5896, 5413, 2212, 130, 3140, 9045, 18467, 18992, 8532, 3182, 17450, 18069, 12891, 2029, 8082, 25, 15975, 8404, 948, 3204, 6829, 1551, 1224, 14230, 13165, 250, 7106, 13755, 18697, 2618, 15761, 12971, 7383, 4756, 8257, 6474, 16615, 18525, 17581, 775, 2963, 1787, 19133, 9441, 16178, 13104, 8900, 13326, 1117, 16465, 6080, 17399, 9585, 1605, 16337, 14062, 16744, 3305, 9770, 18496, 18677, 15474, 19253, 3075, 8997, 19069, 10689, 13315, 18610, 11748, 8626, 35, 10522, 17386, 14307, 2106, 10155, 14335, 7993, 10044, 3505, 10406, 6883, 18429, 15473, 8703, 2764, 2762, 11817, 13090, 3275, 769, 14624, 9988, 15180, 2926, 14169, 18476, 10709, 34, 17284, 12843, 9899, 11255, 14355, 19467, 32, 1041, 17735, 7822, 17992, 1506, 5858, 13155, 15526, 7978, 16745, 14401, 17879, 7492, 16249, 9266, 8259, 15729, 15880, 5717, 1179, 4239, 9458, 11625, 18292, 8870, 14206, 12314, 17662, 6919, 4593, 12113, 756, 11242, 17748, 1706, 3106, 10663, 8700, 13507, 15133, 9306, 15107, 14491, 10987, 11804, 224, 13400, 9078, 5458, 7397, 18723, 14652, 18162, 13511, 3780, 2242, 12598, 5555, 18842, 8519, 12537, 15843, 8759, 10612, 17524, 17029, 6509, 16632, 1496, 12359, 15626, 1867, 6529, 2300, 2457, 2400, 2548, 7391, 3692, 11320, 3578, 4359, 8281, 15297, 5715, 16594, 12051, 15705, 8068, 269, 10644, 15171, 6236, 615, 15407, 18268, 18731, 2896, 11932, 13331, 9940, 14290, 11301, 4292, 8629, 13901, 16395, 15741, 17611, 3329, 3312, 6564, 9725, 12368, 3507, 15341, 6991, 12725, 16616, 7361, 11232, 10530, 14867, 16445, 15084, 6648, 15239, 11743, 5665, 17754, 338, 19328, 13265, 1650, 7139, 6929, 18035, 8181, 8268, 11009, 7272, 4423, 13704, 1353, 14271, 12737, 18531, 14774, 430, 19234, 10646, 17738, 15144, 8574, 9907, 16813, 12132, 14172, 2662, 16973, 9955, 10181, 683, 12919, 16783, 13368, 14462, 12429, 3503, 14551, 13121, 14324, 15753, 19088, 15268, 9669, 18078, 3443, 16222, 6241, 1660, 11746, 7508, 16760, 7338, 12189, 16017, 16496, 14413, 17425, 11920, 12102, 16200, 12594, 6185, 3498, 5718, 1896, 2145, 1074, 17810, 4355, 4535, 1465, 6456, 12987, 13429, 7542, 17801, 6737, 11600, 14710, 1447, 7770, 16505, 10820, 5940, 9536, 2118, 10082, 3257, 18976, 16209, 7655, 14492, 19185, 16975, 14783, 7230, 6519, 6228, 10355, 7288, 5512, 14105, 12870, 16580, 18974, 15048, 12679, 18191, 953, 16436, 7436, 10697, 4676, 11187, 13212, 2347, 5572, 4724, 16710, 15007, 6846, 1459, 9675, 3194, 15355, 7574, 9822, 4818, 12313, 2144, 9239, 3911, 18468, 7098, 1687, 3186, 8332, 10851, 8106, 3410, 14569, 2723, 2783, 184, 8733, 1845, 18955, 6657, 5772, 19338, 5174, 10691, 13260, 15527, 1021, 16481, 14850, 613, 10498, 6242, 2933, 18568, 11029, 13562, 14071, 7044, 15033, 9488, 9185, 880, 9279, 12207, 9317, 18117, 4762, 16129, 14161, 4404, 501, 13420, 15338, 9046, 14001, 12016, 4021, 5338, 10859, 11645, 4866, 7838, 16267, 2172, 11120, 17499, 16533, 14965, 13163, 13957, 13191, 13105, 13215, 6630, 13290, 4918, 1779, 11681, 6181, 6696, 12338, 9378, 18673, 1683, 13404, 12426, 13576, 12494, 14081, 14741, 459, 3143, 10073, 7965, 17357, 17313, 5968, 14663, 12344, 8944, 10461, 18676, 8517, 334, 5227, 6743, 5231, 5730, 13971, 5535, 10325, 9406, 15847, 14012, 10405, 3783, 14875, 4496, 6223, 11198, 277, 9582, 18348, 15298, 1346, 13643, 14074, 16103, 11103, 5302, 1209, 5163, 16550, 1031, 6715, 12751, 36, 18025, 15805, 8196, 9833, 4117, 10250, 10488, 4234, 18091, 3130, 14038, 9423, 13484, 8258, 9073, 15337, 504, 1899, 14562, 15191, 18235, 11537, 1341, 14577, 12673, 8377, 10337, 12883, 9103, 18772, 662, 19217, 4034, 8240, 5960, 14509, 7212, 11128, 18647, 14029, 15959, 14571, 10659, 783, 3678, 9730, 282, 19178, 1596, 16366, 451, 12769, 1442, 8990, 13275, 12420, 261, 7585, 14229, 3464, 5937, 5099, 9778, 2271, 6560, 1815, 9453, 548, 4938, 18458, 8967, 7152, 9817, 4457, 3871, 950, 14545, 7802, 13532, 12412, 8991, 18812, 8523, 14615, 13945, 10281, 822, 10384, 6880, 4250, 6962, 15242, 17423, 5057, 2064, 12071, 17613, 10478, 8971, 11122, 18912, 4658, 14056, 10490, 3866, 16869, 16492, 12515, 5773, 168, 8426, 4301, 12434, 9959, 14138, 2943, 19250, 1659, 11848, 9906, 17971, 14843, 8324, 2078, 11611, 6511, 5428, 6052, 6173, 12290, 15614, 2367, 1639, 8833, 4690, 1743, 9574, 8747, 8041, 18393, 6966, 9447, 9820, 15558, 7848, 5581, 19407, 3534, 7072, 6355, 9208, 3431, 11119, 15896, 10849, 11605, 12738, 14093, 419, 19019, 5298, 11357, 5991, 19073, 5815, 14426, 10228, 7067, 14664, 16612, 1455, 9362, 9901, 13339, 13923, 16940, 2542, 13062, 13478, 376, 19452, 18808, 16341, 8363, 15649, 7929, 8002, 13760, 10720, 3328, 17915, 8720, 10723, 7740, 18657, 12475, 1838, 5410, 3458, 5693, 16898, 6275, 16196, 9138, 8926, 9139, 9878, 13938, 10224, 19155, 14441, 15150, 9152, 5017, 10292, 12413, 108, 4023, 2638, 19189, 9376, 2255, 15167, 7717, 13342, 4849, 18368, 5837, 6480, 15091, 5029, 726, 9796, 1434, 14122, 10427, 18516, 6254, 13695, 15311, 8198, 12458, 13033, 2722, 16347, 4297, 16273, 6727, 13494, 13240, 6028, 11292, 16185, 16714, 17520, 11867, 11889, 4154, 18101, 4094, 7722, 1657, 11359, 14819, 15585, 16833, 2565, 16879, 14847, 5496, 8917, 6202, 4240, 9323, 15888, 17791, 15677, 10822, 1106, 8638, 14647, 699, 7422, 1308, 15281, 17852, 2535, 11018, 11903, 8242, 10818, 15881, 5392, 18184, 7166, 13708, 13919, 2157, 16795, 5605, 4207, 12172, 2370, 14291, 1642, 14475, 2686, 11325, 17361, 9118, 19153, 16671, 4813, 18059, 18680, 218, 8158, 11693, 11454, 16900, 17275, 10615, 13539, 6792, 9504, 17914, 11112, 11592, 16081, 7016, 5811, 5081, 18102, 12031, 6344, 13210, 11640, 10143, 1397, 7682, 10864, 18515, 3436, 4501, 8892, 1858, 19078, 1957, 13483, 8685, 11227, 9900, 15972, 16415, 2294, 5518, 18978, 8420, 17487, 11401, 7561, 19151, 18721, 10470, 11662, 12646, 13034, 7235, 16295, 10840, 18659, 6913, 364, 13489, 5228, 2141, 8043, 7337, 1404, 18073, 2831, 2318, 3244, 8162, 10054, 15742, 8486, 13834, 14096, 9226, 493, 15136, 8193, 2816, 1419, 553, 14119, 3060, 5234, 7232, 14591, 12187, 10189, 7662, 12736, 2508, 1825, 11661, 17572, 935, 17493, 7258, 5084, 10202, 3454, 19384, 18129, 11563, 13921, 18204, 640, 6733, 12468, 10915, 14853, 16221, 10617, 6096, 1854, 18267, 5886, 9198, 18452, 10322, 16754, 10577, 1023, 13868, 11423, 13572, 5680, 2807, 6149, 2235, 9949, 14722, 12985, 18414, 7085, 9260, 11913, 16715, 5529, 17847, 14771, 11962, 16764, 6111, 8006, 10628, 7471, 8536, 14130, 14231, 19438, 5388, 7414, 16336, 18430, 715, 18860, 10682, 453, 8581, 5325, 6090, 17842, 3175, 6944, 17252, 13578, 13012, 7381, 13325, 17274, 17666, 12972, 17890, 9977, 3892, 7280, 5990, 18409, 3827, 7006, 1587, 19095, 7720, 9511, 810, 10760, 18565, 3632, 17946, 6356, 13693, 2634, 98, 16013, 11612, 3734, 6538, 12752, 18957, 12560, 6204, 1388, 12419, 6188, 10632, 14110, 8804, 1099, 17728, 9617, 1524, 6542, 13117, 954, 8219, 7967, 2856, 8121, 12192, 5483, 6745, 12753, 19337, 2007, 1296, 17334, 2484, 10916, 10885, 11632, 15126, 1862, 8848, 10806, 15734, 6520, 13662, 12888, 7294, 58, 16824, 6785, 8666, 6257, 2770, 6908, 8467, 6197, 14240, 12048, 13942, 11004, 12354, 14481, 5138, 11961, 5989, 16929, 11877, 4160, 10546, 18484, 14392, 1467, 17244, 10324, 1514, 5448, 1714, 12832, 8906, 9840, 17175, 852, 8431, 19213, 11747, 12685, 8380, 6072, 12618, 5030, 7377, 6238, 3246, 19146, 16794, 5083, 3084, 1392, 6822, 18965, 9186, 16207, 18967, 7240, 13534, 11626, 15445, 8689, 2150, 16062, 995, 18335, 16761, 2191, 2095, 14844, 101, 11691, 5025, 11832, 11756, 3492, 2202, 15001, 12624, 6097, 13906, 8090, 13815, 12869, 10966, 8215, 6612, 10200, 6708, 16670, 5280, 2441, 1767, 11214, 3961, 18831, 9165, 5016, 635, 7298, 12693, 9766, 6272, 15330, 5261, 4843, 4024, 17723, 12035, 10107, 9650, 13211, 147, 12732, 3009, 2588, 10891, 10569, 11863, 3174, 4587, 6686, 14918, 521, 8345, 3516, 3808, 12460, 6531, 15555, 18343, 12976, 14305, 4709, 15269, 417, 466, 16076, 11779, 7753, 634, 17490, 17612, 12936, 10728, 16488, 13892, 13996, 7507, 3564, 461, 13676, 15426, 3085, 18192, 10360, 15212, 16419, 17627, 12775, 7213, 3546, 18807, 10349, 4789, 3131, 12983, 14952, 16224, 14226, 13953, 11984, 18203, 1072, 19232, 8577, 7830, 2598, 11389, 5060, 8054, 16364, 3760, 17122, 12015, 11692, 17870, 18150, 1175, 938, 10591, 6454, 14728, 10783, 11390, 11291, 15096, 19347, 16998, 6303, 4098, 12890, 8637, 9877, 2967, 7881, 7382, 14191, 11755, 3992, 6418, 18635, 13880, 11523, 4471, 17462, 19455, 10609, 7868, 8309, 11682, 16231, 6628, 5235, 18851, 17727, 16236, 17828, 6893, 5301, 7297, 2842, 8191, 13205, 17533, 2671, 17871, 13836, 14209, 16969, 523, 4613, 5055, 9863, 12586, 13356, 3242, 10350, 19, 8682, 14869, 1294, 17079, 12301, 12037, 3453, 11416, 216, 3749, 5282, 3947, 3919, 14298, 6747, 13085, 7748, 15142, 5278, 12868, 17177, 12287, 5096, 8808, 10235, 13961, 80, 14879, 11110, 1279, 18598, 183, 5254, 9856, 11215, 10528, 16572, 4450, 7256, 7799, 17509, 4817, 15202, 2384, 497, 10178, 3120, 14028, 8868, 15661, 2493, 17152, 7388, 3456, 14188, 8801, 2507, 4914, 14, 14126, 15599, 19309, 5157, 16749, 7549, 14265, 14912, 4537, 7672, 19138, 15895, 1972, 18705, 16608, 10205, 13558, 8583, 7076, 3494, 10930, 19410, 4443, 17565, 5284, 6377, 8996, 5595, 8340, 13397, 4581, 17619, 11051, 16346, 4748, 10890, 16757, 8379, 16393, 17155, 12211, 3690, 3389, 985, 14432, 9740, 1149, 12506, 1047, 732, 2399, 3076, 6979, 12858, 14861, 11366, 12504, 5349, 13994, 13106, 2851, 2042, 4186, 7429, 18681, 12764, 14739, 16605, 16250, 1449, 6942, 4971, 632, 14593, 5882, 6777, 6290, 18076, 1920, 15160, 9248, 6175, 9516, 4492, 304, 17148, 11752, 12974, 18917, 14829, 13537, 9653, 8886, 15969, 11037, 11767, 11841, 11856, 13819, 12441, 14189, 13660, 10616, 2965, 2077, 6064, 14090, 16015, 13230, 18483, 9589, 2252, 13475, 12279, 16741, 1311, 15000, 17919, 18541, 864, 7395, 12907, 5568, 4156, 3979, 18459, 13321, 17821, 1637, 18686, 2620, 5062, 16160, 18788, 13520, 1528, 1212, 1708, 15782, 2313, 18338, 19308, 14145, 6673, 15395, 10151, 17958, 10062, 8436, 12219, 15072, 16986, 14536, 6085, 13424, 17123, 5800, 3817, 12231, 918, 4273, 12568, 19470, 9281, 4260, 7977, 5712, 18005, 527, 11123, 17704, 14418, 16386, 17434, 7771, 19400, 16424, 1768, 15823, 15604, 4087, 17187, 11084, 1389, 1340, 15416, 19267, 13182, 624, 13254, 16584, 16367, 18870, 17875, 13194, 13426, 9541, 11899, 9418, 13500, 6130, 2234, 1678, 1438, 18798, 12510, 2517, 16010, 8662, 2016, 16037, 1332, 17038, 43, 6901, 8468, 1132, 19321, 8734, 17731, 13937, 697, 693, 12573, 17156, 149, 2439, 10306, 18356, 12092, 8079, 14596, 4562, 3138, 3709, 14327, 3565, 11840, 16102, 3053, 8172, 4912, 19238, 14661, 2626, 5760, 17486, 10414, 15822, 13367, 15570, 3402, 1198, 3191, 3163, 7316, 11850, 15837, 14381, 11365, 6680, 6753, 6409, 14681, 1354, 18440, 784, 7541, 1936, 3711, 3881, 10105, 9857, 177, 19325, 12542, 16846, 16203, 14204, 14676, 14690, 16322, 14005, 2350, 1795, 1634, 11557, 18737, 10944, 9370, 7118, 18820, 18349, 4485, 5035, 5264, 7352, 8074, 4932, 1509, 4101, 3540, 12152, 12384, 3129, 14870, 12203, 12167, 11529, 12349, 9869, 8311, 2351, 6953, 7792, 2503, 11905, 9373, 7716, 7764, 4295, 17767, 4913, 14733, 4763, 5635, 2149, 5102, 11731, 11577, 11096, 8319, 2910, 4135, 14610, 18693, 3647, 6423, 16168, 11369, 13403, 16516, 7238, 7870, 2084, 5749, 3238, 7054, 16765, 6801, 16871, 1861, 15767, 10447, 6339, 932, 14531, 11601, 11148, 13590, 14400, 3249, 15568, 18876, 2275, 5172, 8401, 11239, 10418, 526, 14648, 17518, 16587, 2231, 4171, 9874, 7004, 15824, 6389, 5457, 9126, 9132, 18021, 14450, 10902, 4585, 8538, 9613, 400, 2269, 15779, 8488, 16849, 18128, 8276, 9498, 1885, 17373, 15842, 14529, 18355, 15037, 2538, 7407, 16012, 11125, 16775, 12948, 12459, 6578, 4180, 18821, 10397, 18811, 13337, 14619, 4356, 8864, 2306, 3258, 17447, 15907, 9894, 6156, 17833, 15320, 16984, 11800, 11079, 1690, 1814, 16645, 2974, 3046, 5507, 2143, 1121, 6669, 7439, 8542, 15384, 13418, 12838, 6417, 7964, 8270, 14524, 16348, 6248, 8396, 17506, 12456, 5774, 8163, 8722, 10502, 13115, 18507, 8774, 8508, 15117, 5306, 1490, 54, 1417, 3382, 2569, 7425, 3018, 12650, 11624, 2022, 1869, 9151, 5618, 396, 9473, 7459, 10912, 214, 9798, 16174, 7923, 7545, 561, 11813, 15874, 17443, 337, 8806, 5177, 15854, 2953, 4670, 8697, 15415, 688, 6714, 6212, 8359, 13371, 6848, 16486, 8131, 14220, 136, 6914, 16876, 447, 9215, 8580, 18043, 7851, 8719, 7001, 10892, 7415, 14978, 18529, 18219, 15134, 1540, 7451, 16130, 659, 10242, 7913, 4705, 7777, 7441, 10673, 349, 420, 2179, 12925, 17379, 755, 11740, 8567, 1688, 7228, 3334, 15833, 3091, 2552, 3521, 12552, 5184, 316, 17695, 2339, 10833, 15948, 11144, 14586, 15186, 15785, 7800, 7766, 5742, 13377, 2297, 15267, 14941, 10482, 4648, 13040, 14516, 3603, 2790, 3316, 18019, 4837, 7986, 8104, 3445, 6702, 825, 50, 6350, 5103, 6159, 4865, 4976, 12634, 2949, 13598, 13548, 7275, 14612, 10386, 13867, 13848, 9775, 17773, 7262, 10102, 15475, 2777, 17869, 5166, 1916, 17894, 14140, 5590, 17827, 18884, 16863, 11358, 15632, 8980, 18345, 18148, 9385, 11307, 15929, 4951, 7906, 11021, 9813, 5213, 12623, 11680, 3873, 1811, 9290, 11386, 596, 4434, 4557, 1287, 7419, 6973, 9501, 15529, 10308, 13154, 7154, 17021, 6117, 6765, 14827, 11197, 15008, 12841, 6062, 943, 14572, 12316, 7617, 12768, 12637, 2499, 14625, 6147, 12585, 19055, 14471, 13043, 12267, 6342, 12304, 7468, 8940, 9583, 11213, 6108, 10696, 14187, 15083, 17510, 6867, 14328, 16546, 9967, 11673, 17534, 14983, 18773, 12013, 1329, 8188, 4743, 8592, 11542, 3065, 15566, 2817, 12705, 7723, 18364, 2192, 6665, 11305, 9083, 1623, 14261, 4987, 5613, 19214, 15796, 12711, 14731, 9678, 7629, 14429, 4375, 17383, 1048, 11067, 9360, 19204, 15501, 9477, 7651, 426, 16800, 2163, 4488, 6196, 3417, 14944, 18095, 869, 9919, 18033, 11332, 4561, 3162, 10251, 19102, 16258, 14317, 17451, 757, 6471, 13030, 17498, 13547, 206, 10751, 6771, 4092, 5078, 13555, 5196, 6587, 1246, 12665, 3532, 2672, 9951, 9417, 11276, 15245, 166, 9277, 666, 2239, 3617, 4282, 19311, 18041, 12680, 18661, 17633, 13905, 14389, 1697, 17255, 2155, 1184, 6402, 12986, 9349, 8535, 3331, 18971, 18199, 14971, 15588, 3806, 2359, 8151, 10310, 9158, 303, 3121, 13289, 9664, 16522, 12086, 9359, 2864, 2903, 7182, 4722, 5982, 6040, 18653, 8893, 19038, 14927, 225, 6886, 6515, 13681, 7516, 8295, 13791, 17977, 9712, 2514, 6614, 15663, 9157, 14194, 9774, 7505, 7680, 2880, 9086, 9150, 9115, 11143, 1307, 12690, 51, 16403, 13677, 14200, 1273, 5465, 3067, 17217, 7399, 5143, 8838, 12807, 1902, 2782, 11045, 1001, 14285, 3755, 13138, 7276, 5112, 15325, 2153, 18989, 1837, 12278, 18919, 11432, 2625, 17321, 16839, 3132, 12395, 15254, 12308, 12151, 6406, 10111, 660, 10717, 10983, 19442, 18185, 13844, 6093, 7075, 300, 5405, 15894, 15300, 6589, 8895, 1110, 2769, 12900, 2270, 9564, 16293, 2875, 8110, 4982, 18768, 6722, 792, 9649, 15110, 18354, 5118, 15251, 17331, 11491, 11610, 329, 9854, 3073, 14134, 16732, 9621, 6645, 2693, 3935, 5952, 18926, 8026, 11422, 6382, 9994, 12834, 17428, 6703, 11565, 16142, 637, 15169, 17072, 4851, 16812, 13299, 1054, 9092, 14851, 8615, 5046, 5789, 3823, 8511, 2272, 6141, 11845, 6593, 18359, 6025, 7696, 17626, 6713, 5376, 10860, 4120, 12817, 14888, 833, 9939, 18958, 13943, 12683, 15099, 7932, 7339, 4887, 6054, 11982, 10583, 5198, 9462, 6596, 11203, 1486, 13314, 8679, 8993, 1653, 44, 5467, 12317, 16743, 12248, 13197, 7307, 16660, 16286, 6550, 10058, 6057, 3841, 6768, 17974, 15694, 2310, 8611, 10278, 3483, 1030, 17846, 14707, 12100, 10328, 7174, 17150, 12513, 2177, 8128, 13632, 13752, 16447, 17929, 13846, 1953, 17023, 7184, 9654, 9531, 17866, 6884, 7624, 10957, 11684, 15731, 6026, 15736, 1245, 4576, 6410, 9196, 10888, 5560, 592, 18159, 8515, 7301, 16830, 9372, 500, 16335, 4097, 1103, 13362, 11323, 4366, 17686, 1933, 9316, 7530, 17906, 2408, 5471, 2437, 7420, 9630, 319, 5460, 11312, 6811, 19050, 11006, 11686, 12905, 17877, 13519, 645, 10706, 10080, 14386, 13169, 16438, 4182, 6817, 3037, 19313, 18397, 11663, 16798, 1595, 15018, 3850, 2530, 5129, 13226, 13080, 6375, 12864, 9535, 18935, 12776, 14368, 15439, 8664, 13306, 4956, 18746, 5633, 13394, 99, 8792, 5324, 13869, 17046, 14991, 2977, 8810, 6214, 17690, 11724, 1973, 18140, 2488, 13322, 18506, 4409, 12080, 15097, 6146, 245, 7595, 4479, 12917, 14962, 1411, 13652, 13485, 7060, 8520, 15505, 6502, 2469, 17702, 7592, 10464, 14015, 8798, 1431, 16545, 5109, 4405, 4042, 10908, 16291, 11341, 18236, 15443, 12966, 12938, 9116, 6002, 17304, 15978, 8117, 6457, 14718, 2622, 17517, 19064, 10769, 16064, 4416, 1484, 8879, 5489, 3769, 1652, 9053, 6126, 10180, 14662, 3566, 18587, 7110, 1068, 7762, 11477, 4429, 8102, 9706, 12206, 4873, 296, 16686, 17415, 8306, 14896, 906, 8622, 11815, 17779, 10823, 18533, 10785, 7493, 5810, 13467, 8030, 18763, 5318, 15937, 2944, 3766, 6178, 4229, 8053, 17961, 11566, 14762, 15862, 18501, 9402, 3768, 9636, 462, 1607, 9858, 13408, 4745, 9597, 4714, 18346, 9699, 4788, 12746, 13416, 15690, 6177, 11883, 13706, 4831, 8170, 9390, 162, 10649, 8067, 6125, 5358, 2377, 3201, 505, 1477, 7103, 7760, 16244, 16387, 13703, 11451, 17955, 287, 593, 6314, 5631, 7214, 3440, 28, 6483, 1633, 9832, 4664, 12185, 12407, 3957, 13409, 1201, 7953, 4559, 4857, 18615, 8153, 12000, 12365, 18756, 2792, 9705, 17654, 11946, 13637, 8974, 16001, 1436, 11781, 1029, 2544, 2876, 2682, 14775, 16995, 14617, 1336, 10848, 10662, 12767, 912, 7835, 5621, 14478, 12828, 10743, 12548, 3170, 7670, 6440, 2913, 8052, 19441, 7433, 13661, 10472, 3518, 17850, 2568, 10214, 7652, 14752, 12927, 7461, 8231, 8534, 4325, 16478, 18733, 12124, 15373, 14890, 1847, 18433, 93, 4790, 12179, 9905, 6607, 12911, 17188, 15434, 15274, 5616, 3796, 10234, 599, 17625, 5828, 835, 3876, 12866, 9500, 12696, 1364, 623, 8093, 14953, 16105, 5775, 6532, 305, 15820, 19290, 5701, 1930, 4680, 7739, 2419, 10510, 6430, 7002, 13072, 16104, 19065, 13022, 13487, 13531, 14155, 6291, 773, 11507, 7725, 14113, 18892, 3591, 6724, 2232, 13304, 7165, 17390, 2168, 3323, 16165, 7318, 15088, 4235, 9, 15508, 6170, 7784, 18868, 12345, 281, 13312, 4382, 13120, 7506, 18391, 13086, 208, 4054, 15065, 14854, 16107, 8487, 17933, 9616, 6133, 16934, 7523, 18301, 1005, 5503, 440, 8977, 18487, 17286, 10537, 13158, 2325, 7714, 13023, 6514, 18905, 5557, 7598, 10253, 17557, 17997, 15334, 5548, 2237, 9326, 15119, 11235, 5939, 17231, 18084, 2421, 9812, 6421, 14218, 17363, 16925, 3324, 14302, 4330, 3325, 10182, 387, 2526, 734, 8691, 3602, 13724, 7862, 17747, 11633, 15316, 989, 1084, 11960, 3745, 15535, 6843, 9015, 14115, 2547, 10759, 5175, 1820, 8042, 6115, 11299, 14042, 5272, 18980, 1304, 11717, 11737, 5974, 13029, 1559, 5027, 1574, 12956, 16300, 670, 4805, 1312, 6965, 12227, 9865, 17322, 11494, 14373, 16430, 16523, 18648, 19173, 14159, 1711, 12747, 6280, 17976, 9587, 12811, 4605, 6557, 4262, 1295, 8050, 16891, 2549, 15564, 14753, 12536, 5746, 15282, 682, 4513, 4043, 6590, 16466, 15047, 8156, 11801, 1414, 12881, 10996, 16147, 16214, 3951, 14532, 914, 1004, 13900, 19246, 10311, 1269, 821, 15011, 3218, 5956, 17778, 3220, 3552, 10737, 14635, 3366, 6229, 9976, 6690, 16067, 2509, 4286, 9751, 5902, 2151, 15423, 10315, 6892, 2796, 15560, 836, 18168, 14921, 8210, 4628, 18186, 17570, 1108, 3119, 7431, 4338, 4622, 7115, 17319, 2227, 17416, 5487, 7557, 6736, 4899, 11500, 8994, 10873, 6486, 18521, 7521, 18247, 18915, 6279, 16642, 7821, 8943, 19254, 2451, 5880, 16808, 9271, 19261, 5501, 4324, 17786, 18366, 12695, 2423, 11398, 6597, 10772, 17140, 6044, 17370, 4990, 2901, 3028, 7620, 8012, 6820, 1791, 15751, 12098, 17678, 18846, 8723, 8962, 4383, 18704, 15163, 3956, 3099, 2610, 13904, 15662, 6660, 3118, 17003, 16083, 10198, 10870, 18066, 7389, 17707, 8442, 10188, 16972, 3649, 8604, 18537, 18719, 8094, 15158, 11379, 10801, 5583, 614, 17553, 6780, 7780, 16202, 4131, 15085, 17134, 8543, 5993, 4713, 9882, 1974, 12320, 3971, 947, 13433, 12910, 4178, 2925, 17006, 41, 17769, 4802, 16816, 9611, 3770, 13200, 17638, 3569, 9129, 16565, 492, 11289, 15380, 11450, 16639, 19013, 2228, 13235, 5239, 11857, 4847, 2922, 940, 15199, 15809, 15229, 6183, 3226, 8256, 14319, 4033, 6572, 12852, 18444, 13828, 16354, 16021, 976, 17853, 2787, 11382, 13873, 10036, 395, 7490, 16441, 7344, 11495, 806, 2690, 8736, 11623, 3, 10951, 1689, 3714, 18424, 14581, 13764, 4197, 15771, 9117, 14700, 13580, 12375, 1967, 15538, 10844, 580, 11637, 1671, 10686, 5141, 7009, 18151, 3388, 11145, 7795, 17790, 1382, 11696, 12328, 7520, 7579, 2248, 11017, 2331, 16398, 8073, 9862, 17087, 669, 15702, 5795, 12906, 10973, 13091, 18918, 11282, 13269, 12645, 6351, 6925, 325, 15459, 19391, 5492, 18888, 2480, 11885, 17105, 8819, 3735, 7703, 18310, 328, 18121, 17537, 4232, 1764, 2323, 3315, 13541, 5028, 4486, 12324, 18, 1086, 9954, 15105, 14634, 2415, 18111, 4870, 9927, 6432, 9992, 9872, 1999, 1668, 12085, 3752, 15579, 7894, 2866, 10727, 3284, 9633, 17052, 7095, 3896, 6902, 6647, 4126, 9302, 10762, 1435, 15955, 14727, 12958, 13859, 10790, 4853, 17736, 3304, 7966, 11938, 941, 11630, 13419, 12837, 307, 2845, 4822, 13276, 1635, 357, 7101, 13666, 4007, 2119, 15818, 8829, 4687, 14270, 17411, 13893, 19223, 13646, 18010, 905, 7299, 2560, 7976, 17963, 1562, 17922, 11438, 15769, 1066, 1090, 13302, 3321, 2673, 13175, 15183, 13954, 10357, 11539, 16050, 3794, 6855, 13142, 2038, 2428, 15060, 4223, 3117, 19345, 530, 273, 8986, 16606, 4347, 15817, 11074, 8635, 11967, 1135, 3912, 13383, 4233, 14603, 17414, 2241, 13502, 4879, 16144, 17077, 1855, 10244, 16407, 9392, 5340, 6243, 17209, 3822, 15575, 14645, 4545, 3070, 1654, 13490, 17594, 1315, 10730, 2055, 12709, 12923, 10309, 8322, 2878, 5412, 17309, 3550, 10515, 913, 9435, 9463, 11926, 14574, 1251, 8312, 842, 4915, 12273, 2340, 79, 15223, 9224, 12816, 13829, 11463, 13930, 12333, 8378, 18225, 5540, 9421, 6676, 17317, 18241, 6043, 15623, 10366, 6348, 2809, 10213, 13709, 7713, 9429, 7281, 9470, 17574, 1208, 17028, 10291, 3793, 4267, 6854, 1204, 3687, 3272, 11340, 15456, 15660, 6738, 2205, 8576, 5470, 18365, 19414, 16535, 6938, 10922, 8059, 12584, 3318, 18477, 11220, 13076, 9499, 4591, 6385, 16226, 3843, 14295, 1092, 8394, 258, 383, 16628, 14967, 15523, 7850, 4313, 5370, 2858, 2165, 18786, 763, 2208, 16274, 4113, 13606, 9502, 8899, 15326, 15685, 10395, 2296, 1553, 608, 39, 6427, 10017, 16427, 8446, 12945, 348, 9693, 10001, 13657, 14348, 12835, 17873, 14954, 2152, 7581, 2849, 3607, 5128, 2265, 18504, 10767, 7519, 6931, 5475, 12952, 19125, 15113, 1612, 11772, 9094, 10361, 19301, 5741, 5398, 16778, 17989, 18991, 2355, 9183, 6073, 14699, 15909, 9497, 6654, 7756, 9672, 17981, 12962, 13931, 4030, 18262, 9850, 16480, 4819, 2162, 4407, 6804, 9122, 19307, 5269, 306, 9207, 12981, 12444, 10999, 14592, 17118, 8447, 10320, 1124, 3087, 16401, 18758, 2973, 5794, 8877, 2869, 18308, 8421, 14251, 4909, 9486, 6878, 14106, 5352, 4969, 18571, 17062, 13920, 10479, 1134, 17226, 7610, 9510, 6331, 12275, 11351, 5199, 5904, 4125, 14117, 8344, 13125, 8220, 11988, 18079, 19383, 4955, 2109, 6720, 1907, 2431, 13244, 17815, 1713, 9780, 2158, 1176, 2246, 16948, 117, 13503, 8432, 2951, 15490, 9163, 3473, 16217, 236, 964, 10026, 9622, 7470, 14419, 13889, 18320, 4967, 3217, 4758, 10075, 11940, 6316, 7197, 16449, 6045, 8671, 14228, 8789, 6842, 10359, 18906, 1138, 2479, 7846, 2309, 5865, 4829, 14088, 7107, 14638, 3671, 12901, 190, 18671, 19396, 13064, 9387, 8791, 14855, 15982, 8101, 18555, 4002, 1803, 1370, 1555, 8678, 3844, 4238, 9388, 2472, 6050, 9354, 10594, 5654, 18381, 12474, 12410, 11522, 11010, 7804, 13329, 865, 12473, 17647, 2908, 7205, 13977, 13650, 16164, 802, 4242, 9394, 4647, 12845, 18849, 2066, 17855, 14082, 17227, 18042, 16319, 16950, 16931, 8632, 7321, 4850, 2985, 8715, 15263, 6535, 17465, 13636, 5732, 13974, 18594, 11344, 5360, 4284, 18428, 16536, 4158, 17745, 8701, 12530, 15291, 16241, 6137, 17529, 10197, 10185, 4717, 16513, 10764, 2799, 7926, 10995, 6983, 15515, 794, 5061, 12990, 1891, 13622, 18007, 7032, 571, 11518, 4523, 18963, 18032, 17781, 8134, 9802, 846, 8109, 17964, 8620, 7311, 12114, 3144, 3104, 17432, 17389, 11666, 14881, 11251, 17969, 12409, 14961, 18528, 1821, 13006, 11866, 1645, 5801, 15057, 13999, 13027, 9197, 4827, 17862, 7892, 17162, 14513, 10775, 11728, 12040, 7033, 9562, 14174, 17497, 9371, 16339, 2203, 16213, 2139, 14007, 14175, 4159, 13295, 19060, 12379, 16989, 9030, 10536, 5286, 7841, 13648, 13242, 3800, 13634, 6145, 15094, 7225, 8463, 9909, 14920, 18510, 19357, 8612, 19334, 1649, 11902, 7271, 16912, 7204, 2067, 2550, 14872, 15544, 14777, 13735, 3326, 17070, 1739, 4153, 4203, 12180, 11256, 4401, 14089, 8438, 16126, 10978, 12539, 8847, 13434, 7943, 15816, 13969, 8194, 2511, 10589, 4308, 3003, 18321, 2942, 2442, 18205, 5309, 11739, 14437, 9665, 3977, 1784, 17952, 5181, 11702, 2877, 4546, 9320, 5240, 2244, 3765, 13543, 10273, 19387, 413, 5449, 7843, 9282, 9383, 19323, 6980, 15204, 18369, 14284, 15893, 5369, 13528, 8362, 1691, 3278, 1897, 5812, 1057, 1181, 16096, 5424, 6484, 12230, 10742, 3698, 12991, 14364, 14507, 4769, 13160, 6468, 9221, 12614, 6877, 18490, 18108, 7559, 15960, 16495, 718, 11875, 19205, 2424, 12134, 5406, 16112, 17800, 10284, 17419, 12416, 16390, 14378, 9552, 15954, 6024, 11839, 12762, 9457, 5011, 9814, 7255, 16082, 3568, 2545, 18281, 17571, 8283, 12355, 14148, 8248, 9040, 13958, 17088, 2468, 1918, 11508, 5314, 14900, 8084, 13805, 3416, 1207, 5439, 13857, 13560, 6543, 351, 5214, 1450, 13459, 1705, 7998, 16507, 3723, 18119, 5628, 17359, 2466, 9386, 6273, 17351, 15840, 1094, 15956, 1893, 1707, 16539, 15776, 5002, 14344, 3107, 16034, 1322, 13228, 7741, 18499, 8748, 13551, 8469, 11431, 14997, 15356, 11164, 15115, 12862, 6003, 12787, 4481, 15573, 4753, 3778, 5368, 5506, 1292, 7449, 12505, 7685, 4348, 11136, 4008, 7168, 11027, 13879, 4547, 60, 18423, 2189, 10744, 13619, 13231, 5753, 2061, 16769, 14795, 4536, 8027, 16416, 6448, 15425, 14546, 377, 11886, 8762, 10963, 12655, 9728, 6386, 5462, 12485, 9554, 9688, 10854, 15277, 15215, 966, 2835, 7278, 3396, 3169, 6485, 9035, 9777, 10923, 14249, 2068, 7226, 13912, 17856, 3180, 3937, 10711, 2025, 14737, 11064, 13870, 11178, 15323, 17794, 827, 8686, 2010, 13888, 4045, 10258, 15226, 19349, 3210, 10170, 13962, 8478, 11742, 5481, 2364, 999, 1422, 5048, 19382, 597, 15135, 764, 9701, 10702, 3989, 5427, 9363, 12604, 3654, 7836, 5104, 6803, 5153, 16374, 13717, 3034, 14323, 6336, 6528, 18542, 10561, 5321, 8526, 18135, 16999, 3214, 2365, 12298, 6806, 478, 4678, 2841, 17067, 18123, 5209, 4328, 6106, 14181, 14669, 18234, 10576, 182, 13813, 2914, 18173, 4354, 13011, 7871, 11642, 16933, 5645, 1247, 11439, 17541, 14766, 15265, 14256, 3041, 6530, 4958, 15485, 484, 17843, 6030, 8884, 14928, 18796, 11890, 6699, 8135, 16175, 14643, 5122, 10241, 16882, 18056, 5735, 5694, 16705, 16672, 12214, 10685, 14606, 4459, 19188, 3338, 6536, 16239, 11188, 17121, 3059, 9124, 15016, 11931, 8584, 14616, 6123, 15996, 11287, 4524, 18316, 11245, 14179, 17744, 9367, 13274, 14234, 18754, 12969, 9002, 452, 8521, 2521, 17669, 13336, 1895, 11969, 6437, 16310, 9093, 13649, 12514, 2134, 11086, 14237, 9628, 13234, 3913, 1684, 17010, 12779, 12669, 2753, 9036, 4922, 16270, 647, 18261, 10901, 12758, 9525, 11113, 5672, 9786, 11230, 19360, 13850, 4425, 10136, 14375, 5666, 9419, 2863, 4225, 4876, 10112, 18701, 6120, 12918, 16773, 3830, 1098, 13057, 14555, 14422, 19169, 14427, 16085, 4911, 11384, 6799, 10164, 5185, 7584, 5771, 16872, 3542, 2456, 13811, 12118, 13600, 3869, 18950, 3080, 3166, 11228, 4530, 17378, 18492, 6824, 8606, 10466, 18495, 3339, 8235, 6950, 6370, 12398, 11137, 2731, 16937, 3136, 10990, 17845, 9225, 17236, 16893, 19245, 15278, 6623, 14277, 10955, 5133, 4952, 4736, 18543, 12664, 9848, 15108, 10549, 16307, 9034, 12169, 4919, 16198, 17086, 13037, 421, 9920, 11891, 6332, 10331, 14163, 6516, 12727, 15102, 2006, 11627, 1609, 13573, 2194, 5021, 2125, 7729, 1487, 1616, 15621, 6282, 14579, 9408, 17787, 1163, 18360, 5165, 1226, 7430, 17459, 6900, 16204, 15985, 13294, 14519, 982, 8920, 10471, 4508, 8857, 7750, 8778, 13907, 9189, 18550, 16342, 10449, 16577, 11303, 18357, 11855, 10624, 4335, 1666, 19248, 2210, 18817, 18805, 14824, 767, 16817, 1597, 15743, 2609, 7985, 10424, 14264, 10147, 6963, 7815, 1006, 4202, 17402, 15984, 3127, 6369, 10780, 2319, 11413, 18776, 8667, 15327, 12963, 17878, 17392, 14578, 2653, 15273, 2299, 16462, 1174, 7331, 19046, 19286, 1096, 17643, 3467, 19460, 10174, 8113, 6506, 3539, 128, 3533, 17444, 18149, 2589, 13277, 11408, 9341, 11237, 5201, 7855, 9329, 11372, 5559, 7635, 4022, 6847, 15990, 7460, 1032, 4146, 15695, 14760, 15533, 19446, 7163, 9667, 6089, 13365, 2336, 4567, 16994, 6719, 4900, 1504, 12258, 11714, 7834, 9123, 15093, 18493, 19110, 18567, 18039, 4270, 6626, 10018, 278, 15765, 10926, 8161, 14279, 2045, 1575, 5539, 11699, 13620, 7114, 8047, 19335, 14784, 15857, 4927, 3044, 1610, 17449, 10693, 13575, 19016, 928, 17713, 14002, 16544, 10573, 156, 4411, 10129, 3790, 1143, 9299, 3092, 3172, 12534, 14407, 12256, 4454, 15644, 1170, 5144, 17730, 2820, 19120, 12686, 17954, 7102, 4702, 8619, 16003, 4476, 2837, 15586, 3626, 15247, 16227, 2826, 9966, 19331, 494, 5339, 7279, 18911, 18873, 14241, 6048, 14482, 333, 1172, 14974, 13697, 18569, 14587, 10085, 6231, 1469, 2718, 13866, 6237, 9761, 18609, 13887, 19432, 13110, 12032, 5950, 15400, 237, 13536, 18700, 6150, 3256, 15549, 17041, 8874, 18575, 12225, 14498, 14852, 6954, 11315, 6060, 13305, 616, 542, 16049, 14013, 10585, 11778, 9588, 6037, 15246, 9632, 9178, 3634, 8507, 753, 16578, 3943, 18646, 2001, 628, 17761, 7096, 1621, 8495, 1725, 7883, 1753, 4249, 2060, 10095, 14859, 17640, 1870, 14128, 19354, 8428, 19036, 1288, 14095, 12706, 19454, 6247, 5, 12556, 12743, 6195, 7742, 5545, 12057, 6135, 16287, 9639, 16717, 12025, 7690, 1024, 11411, 7951, 11858, 8799, 4934, 19350, 2174, 2675, 12629, 14170, 10560, 2136, 3327, 8930, 15832, 6148, 15493, 6766, 10814, 6180, 3751, 12144, 10763, 8008, 8262, 11793, 8937, 19028, 15986, 8115, 7572, 2870, 13078, 5926, 9311, 7626, 18668, 2540, 11377, 15478, 5146, 2830, 7177, 16990, 8648, 17880, 4503, 722, 14763, 12846, 1482, 4208, 3544, 2304, 1908, 8854, 8029, 15930, 10815, 12216, 8464, 13691, 8800, 16700, 16483, 15231, 3040, 11511, 7312, 18752, 6139, 18165, 5958, 14535, 9426, 10880, 5887, 14247, 11816, 14822, 13990, 5423, 7732, 16829, 13825, 11829, 17418, 5523, 1505, 2334, 8025, 7569, 7500, 10932, 508, 8984, 1833, 4065, 14408, 11258, 9897, 14104, 17689, 16850, 9058, 9866, 539, 602, 10364, 6896, 10454, 5090, 19272, 13004, 19305, 14682, 14097, 9615, 3362, 1829, 15525, 16019, 1865, 7068, 7510, 19022, 8154, 11893, 13979, 6943, 2794, 2080, 13361, 6863, 10071, 7028, 6800, 13613, 17001, 14758, 8238, 11954, 5999, 17575, 14421, 10698, 2031, 17550, 5652, 19058, 2711, 6035, 12253, 12370, 15006, 10158, 14835, 2533, 6168, 4757, 5468, 19430, 18000, 11226, 18907, 17146, 16455, 12937, 19091, 18054, 7216, 14311, 11849, 6428, 9758, 15403, 12653, 2079, 3428, 3205, 1937, 16834, 3700, 9524, 5036, 2223, 19142, 11871, 19128, 1261, 4303, 263, 1192, 9576, 1832, 1938, 8400, 1087, 14142, 14092, 3643, 2972, 11672, 1826, 14814, 5482, 2873, 9679, 16540, 13630, 12877, 5846, 13252, 18131, 609, 17913, 12353, 11838, 17860, 3282, 12383, 7749, 10824, 3400, 15034, 16068, 9012, 13255, 10023, 11533, 11040, 15233, 6297, 4867, 1724, 9427, 11231, 1243, 5677, 19236, 19032, 9936, 8923, 16457, 9602, 3112, 19206, 2586, 3642, 6189, 10898, 3344, 5474, 7974, 14227, 18631, 10788, 18614, 1173, 8174, 10115, 672, 3032, 10750, 5256, 76, 323, 15156, 11790, 3885, 3915, 5563, 6076, 14828, 6798, 268, 2709, 15166, 5844, 5630, 9484, 9793, 19378, 13166, 14149, 9314, 9006, 254, 8337, 9011, 3573, 1073, 3213, 14280, 15046, 5381, 6289, 9734, 8627, 11705, 19358, 8168, 4388, 3167, 10007, 6447, 4072, 5596, 13816, 4453, 8775, 17277, 13553, 4611, 4385, 7038, 7269, 19093, 3831, 18959, 10297, 2213, 15676, 9830, 5476, 11770, 1618, 8229, 11209, 16527, 17718, 11480, 10991, 5653, 7842, 6507, 15255, 10857, 456, 3596, 16402, 4142, 18050, 1844, 13581, 11656, 11464, 16225, 6573, 10652, 3898, 9964, 11294, 468, 7017, 14338, 14796, 5638, 17803, 19112, 19242, 5943, 326, 12431, 15285, 5333, 11652, 17692, 14553, 7151, 1941, 16592, 18562, 1894, 16634, 18520, 10664, 3646, 17147, 9481, 8905, 13918, 11474, 10060, 3530, 10513, 12300, 17293, 4517, 10144, 16897, 3465, 2169, 8462, 11556, 18588, 6754, 17141, 18077, 7654, 6118, 7309, 7956, 9571, 361, 7078, 1722, 2895, 17500, 17512, 5249, 3788, 15420, 16704, 17956, 17318, 7227, 12967, 7375, 17937, 16910, 17888, 8243, 11206, 7728, 15950, 4175, 7126, 1158, 15261, 8853, 14925, 15387, 17421, 18030, 5579, 1548, 7208, 3928, 11935, 12532, 406, 18438, 14158, 4190, 9794, 2024, 12527, 7378, 10753, 14554, 2692, 3839, 729, 12174, 14283, 18856, 14598, 8545, 15940, 17391, 5312, 1977, 17766, 19247, 16633, 10067, 564, 5928, 10886, 2204, 11864, 3353, 286, 1547, 4346, 18023, 16877, 7910, 14050, 9950, 10323, 8972, 18388, 19362, 10108, 7858, 6934, 19041, 12788, 5708, 17732, 5446, 4977, 5610, 3940, 1407, 1806, 11659, 13820, 7147, 4836, 17918, 556, 18753, 19233, 15590, 15794, 5361, 2222, 2117, 14809, 11223, 12744, 6729, 3728, 19068, 16979, 17116, 10237, 12294, 2739, 17190, 1871, 12249, 4953, 6302, 18518, 14924, 10863, 3905, 11530, 16958, 13616, 11597, 9079, 13281, 11688, 18688, 12136, 18259, 5456, 9847, 6831, 11676, 10971, 14914, 11771, 17276, 9398, 10199, 16361, 9149, 5332, 4643, 2040, 4256, 16493, 15208, 16637, 15078, 13020, 4552, 1792, 11695, 3870, 9913, 8205, 15336, 3486, 10126, 17531, 14139, 7604, 18981, 4688, 19266, 10613, 903, 16965, 3452, 17959, 17573, 16613, 12374, 8716, 6581, 5888, 13199, 16982, 6588, 7616, 6420, 5308, 6556, 14993, 9891, 6253, 14500, 4189, 19439, 47, 15127, 7524, 2592, 9285, 311, 13472, 16707, 10811, 858, 4693, 18924, 12262, 9496, 6074, 13417, 4532, 5275, 19209, 8908, 17473, 11861, 11751, 8731, 16188, 10239, 10786, 17047, 129, 3055, 555, 15394, 11034, 15532, 17816, 3624, 10194, 8180, 5580, 7242, 6772, 12810, 9580, 2971, 18480, 7024, 19377, 6493, 8293, 10877, 1313, 15781, 12091, 6312, 17181, 4191, 5491, 11708, 10028, 16548, 9937, 6577, 6323, 3763, 2166, 16215, 8643, 16698, 2446, 1961, 3593, 6999, 7313, 15369, 3814, 7183, 17841, 13455, 13949, 2111, 12400, 9648, 14108, 10243, 15377, 6650, 15774, 12989, 17068, 3052, 375, 7511, 8147, 13053, 1801, 14618, 9733, 11725, 11853, 8770, 17294, 14051, 5624, 11915, 2176, 11844, 7638, 8772, 2836, 18098, 19123, 2483, 4543, 9726, 13688, 3702, 2712, 14306, 16894, 14203, 9984, 13113, 7642, 4049, 13978, 8265, 1817, 18774, 2936, 12341, 8568, 1760, 5076, 14022, 1348, 18106, 16363, 15542, 7049, 8851, 16090, 8824, 19030, 11257, 2090, 8894, 11011, 2262, 14834, 15524, 3829, 16618, 16327, 17820, 8173, 13273, 7469, 3438, 786, 10436, 9605, 9691, 6945, 16664, 605, 11455, 16785, 2993, 10841, 1338, 17133, 14316, 412, 14798, 16246, 5698, 6164, 18250, 13718, 4128, 752, 2049, 6709, 4826, 4421, 4770, 5606, 1362, 17172, 17327, 19244, 19255, 6496, 11830, 1310, 7048, 187, 265, 18110, 6790, 15786, 917, 3243, 18389, 8189, 5296, 15780, 16790, 18242, 8132, 6056, 4299, 617, 14162, 1352, 16640, 3686, 2405, 7573, 14453, 17505, 15066, 2249, 3151, 8429, 6259, 3907, 8807, 3618, 5544, 11024, 17037, 2037, 15775, 929, 4028, 12904, 2663, 8540, 15090, 11971, 11639, 13809, 2687, 178, 7666, 7885, 17829, 5288, 16171, 16269, 5395, 642, 1841, 8253, 7402, 17254, 15313, 18687, 8177, 229, 10933, 4068, 18170, 12141, 9274, 14337, 8046, 14781, 14195, 9448, 10940, 6230, 7444, 8218, 12047, 6912, 15401, 13354, 3936, 4281, 15627, 19099, 2982, 12164, 14190, 1300, 14405, 15594, 8105, 12137, 11306, 6730, 10090, 2882, 1554, 17597, 15500, 13841, 2099, 926, 8863, 17836, 16389, 13586, 5627, 3058, 12266, 9771, 5552, 7754, 18220, 19239, 8209, 8876, 6069, 7497, 5674, 11685, 16756, 7090, 16676, 6082, 2879, 2410, 15686, 3005, 8374, 996, 12432, 9544, 12797, 13388, 797, 15236, 11419, 5347, 11908, 4752, 2298, 6826, 7686, 12677, 7827, 4641, 2293, 9581, 11799, 13728, 5681, 14336, 15648, 10508, 5387, 15421, 2654, 9618, 5514, 17854, 7241, 2865, 3683, 9284, 16303, 13853, 16814, 15012, 7509, 18512, 6779, 6569, 3502, 5740, 8370, 15462, 14670, 10625, 17696, 371, 8633, 6264, 9522, 7537, 1929, 9560, 18929, 17990, 9055, 13223, 3310, 2991, 11780, 3225, 4675, 8133, 17706, 18595, 15125, 16517, 3926, 13603, 3525, 7371, 15124, 12389, 9924, 7027, 15476, 15029, 9170, 1118, 11020, 2725, 11606, 18558, 1403, 16617, 1533, 13645, 9312, 4526, 2927, 7904, 6213, 1114, 7661, 3890, 16924, 15997, 4469, 11000, 13687, 1922, 11921, 8867, 16380, 17642, 14359, 1546, 16179, 5247, 11906, 8477, 13389, 12873, 4387, 9020, 15452, 18416, 524, 12697, 4415, 17530, 14915, 19130, 4883, 6359, 13327, 19136, 5193, 15666, 3791, 9494, 13721, 7203, 7171, 18461, 4984, 8889, 5834, 2012, 2023, 5098, 13812, 18072, 4773, 644, 2763, 15391, 9301, 1280, 9193, 5689, 3490, 5026, 15139, 18249, 9264, 8934, 850, 7454, 2791, 8953, 567, 2183, 10623, 17797, 114, 7191, 3227, 5756, 7527, 7273, 3160, 17015, 15744, 5159, 11236, 14595, 1952, 10704, 14706, 10, 8590, 13615, 8140, 9203, 14837, 220, 18977, 4224, 11234, 13902, 5766, 14216, 19376, 1762, 3202, 12448, 1065, 3993, 6445, 19449, 18126, 18727, 10962, 5455, 13955, 2724, 18936, 18539, 8273, 13149, 13496, 188, 2756, 6583, 14006, 1423, 19351, 10807, 11524, 15379, 7775, 11678, 17397, 12651, 10104, 17061, 6890, 18238, 14257, 2806, 19283, 10948, 19007, 5765, 4470, 17218, 9772, 10725, 17814, 342, 15578, 15856, 2578, 7794, 7797, 17657, 3727, 18100, 16456, 14745, 3637, 16208, 17435, 6176, 17966, 3551, 2546, 443, 19312, 10614, 6460, 10552, 1229, 8066, 17112, 10572, 7783, 5222, 11134, 12798, 8936, 8578, 16255, 12722, 18651, 6403, 6533, 18133, 13713, 2666, 18961, 8433, 16206, 1537, 5086, 15038, 5779, 16132, 10621, 16000, 12992, 7921, 1, 5864, 19277, 4542, 18605, 15220, 3927, 18171, 6266, 7046, 7747, 3922, 3165, 3900, 14383, 8351, 10192, 15550, 18734, 19170, 10626, 11015, 7710, 11345, 444, 11727, 14448, 11202, 3906, 3261, 9037, 7059, 7618, 4711, 12226, 1466, 2124, 5469, 1819, 1270, 9137, 6293, 5758, 1380, 4064, 626, 8139, 69, 5259, 15062, 3413, 4893, 14352, 6635, 14121, 18882, 5972, 17312, 5311, 4715, 1492, 13440, 10440, 7600, 969, 5077, 6636, 8660, 3047, 7812, 9091, 8213, 3694, 11058, 10747, 4210, 13910, 9413, 11753, 7735, 18706, 3221, 2795, 5206, 2824, 8200, 15726, 52, 7757, 11448, 13964, 5267, 13446, 7517, 8699, 12590, 9130, 8015, 4553, 174, 13617, 15992, 2388, 8992, 10394, 8727, 17222, 18834, 11732, 10638, 3176, 17539, 3289, 9624, 18998, 5315, 15834, 13407, 1375, 14561, 3254, 13359, 12765, 12390, 8254, 393, 11355, 124, 16452, 6986, 14677, 17772, 11789, 4649, 13770, 17972, 5550, 16153, 11254, 706, 12148, 6918, 10283, 4133, 2715, 18777, 8279, 4185, 12130, 680, 11430, 9568, 8562, 18103, 10453, 10419, 6292, 18532, 4978, 2797, 19202, 15801, 3584, 5542, 14767, 15410, 19409, 19465, 19392, 977, 7380, 10658, 14810, 3027, 5619, 547, 14717, 3640, 14528, 9475, 2705, 2487, 4998, 16881, 12083, 13832, 10881, 887, 16212, 13427, 1441, 6326, 2635, 15829, 11158, 7105, 10381, 13950, 2002, 16885, 13249, 15675, 4930, 3838, 11085, 6740, 17823, 17753, 16739, 2840, 1926, 12188, 9652, 14036, 18065, 16658, 8505, 5531, 6544, 5716, 15608, 2100, 7393, 12437, 4107, 2288, 17719, 8929, 13941, 13725, 14552, 1879, 15693, 2522, 5485, 2606, 4310, 10321, 18732, 2968, 11579, 176, 19303, 8329, 4646, 13272, 8796, 465, 2789, 12833, 1171, 9075, 15052, 9670, 14362, 1205, 8546, 428, 12569, 3290, 7917, 16541, 1840, 758, 1563, 13984, 8031, 2259, 983, 2559, 3036, 3623, 12281, 9526, 14863, 17793, 5906, 12417, 3156, 16247, 4336, 12240, 11047, 4426, 14087, 9971, 13751, 14904, 9720, 10989, 13807, 17057, 17831, 10964, 13198]

## Inference

In [ ]:
import time

ix = [0]

correct_predictions = 0
total_predictions = 0
total_inference_time = 0

# set model to evaluation mode
model.eval()

for i in ix:
    row = df.iloc[i]
    state1 = row['trimmed_state1']
    state2 = row['trimmed_state2']
    # map the human classification to the labels that the model will predict (1 = clone/near duplicate & 0 = distinct)
    actual_class = 1 if row['HUMAN_CLASSIFICATION'] == 1 or row['HUMAN_CLASSIFICATION'] == 0 else 0

    # preprocess the input data
    inputs = preprocess_for_inference(state1, state2)

    start_time = time.time()

    # run inference
    with torch.no_grad():  # disable gradient computation
        outputs = model(**inputs)

    end_time = time.time()
    inference_time = end_time - start_time
    total_inference_time += inference_time

    # extract logits and apply softmax
    probabilities = torch.softmax(outputs.logits, dim=-1)

    # predict the class with the highest probability
    predicted_class_id = probabilities.argmax(dim=-1).item()

    print(probabilities[0, predicted_class_id].item())

    is_correct = predicted_class_id == actual_class
    if is_correct:
        correct_predictions += 1
    total_predictions += 1
    if total_predictions % 1000 == 0:
      print(total_predictions)
      print(f"Accuracy: {correct_predictions / total_predictions:.4f}")
      print(f"Average Inference Time: {total_inference_time / total_predictions:.4f} seconds\n")

print('===============DONE===============')
print('Current Represenation: ', current_representation)

accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy:.4f}")

average_inference_time = total_inference_time / total_predictions
print(f"Average Inference Time: {average_inference_time:.4f} seconds")

0.9970136880874634
===============DONE===============
Current Represenation:  content
Accuracy: 1.0000
Average Inference Time: 1.4481 seconds
